## Initialization  

In [1]:
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:98% !important; }</style>"))
%load_ext autoreload
%autoreload 2

import os 
import sys

print(sys.path)
import time
import argparse
import yaml
from tqdm import tqdm, tqdm_notebook, trange
# import tqdm.notebook.trange as tnrange
import copy, pprint
import numpy  as np
import torch  
import pandas as pd
import torch.nn.functional as F
from torch.utils.data import DataLoader 
import scipy.sparse
from time import sleep
from scipy.special import softmax
 
from datetime import datetime
from GPUtil import showUtilization as gpu_usage
 # from tqdm import trange, tqdm
from tqdm.notebook import trange, tqdm
sys.path.insert(0, './src')
from envs.sparsechem_env_dev import SparseChemEnv_Dev
from utils.sparsechem_utils import load_sparse, load_task_weights, class_fold_counts, fold_and_transform_inputs, print_metrics_cr
from dataloaders.chembl_dataloader_dev import ClassRegrSparseDataset_v3, ClassRegrSparseDataset, InfiniteDataLoader
from utils.util import ( makedir, print_separator, create_path, print_yaml, print_yaml2, should, 
                         fix_random_seed, read_yaml_from_input, timestring, print_heading, print_dbg, 
                         print_underline, write_parms_report, get_command_line_args)

 

print(' Cuda is available  : ', torch.cuda.is_available())
print(' CUDA device count  : ', torch.cuda.device_count())
print(' CUDA current device: ', torch.cuda.current_device())
print(' GPU Processes      : ', torch.cuda.list_gpu_processes())
print()

for i in range(torch.cuda.device_count()):
    print(f" Device : cuda:{i}")
    print('   name:       ', torch.cuda.get_device_name())
    print('   capability: ', torch.cuda.get_device_capability())
    print('   properties: ', torch.cuda.get_device_properties(i))
    ## current GPU memory usage by tensors in bytes for a given device
    print('   Allocated : ', torch.cuda.memory_allocated(i) ) 
    ## current GPU memory managed by caching allocator in bytes for a given device, in previous PyTorch versions the command was torch.cuda.memory_cached
    print('   Reserved  : ', torch.cuda.memory_reserved(i) )   
    print()

gpu_usage()                             

['/home/kbardool/kusanagi/AdaSparseChem', '/home/kbardool/miniconda3/envs/pyt-gpu/lib/python39.zip', '/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9', '/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/lib-dynload', '', '/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages', '/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages/IPython/extensions', '/home/kbardool/.ipython']
 Cuda is available  :  True
 CUDA device count  :  1
 CUDA current device:  0
 GPU Processes      :  GPU:0
no processes are running

 Device : cuda:0
   name:        NVIDIA GeForce GTX 970M
   capability:  (5, 2)
   properties:  _CudaDeviceProperties(name='NVIDIA GeForce GTX 970M', major=5, minor=2, total_memory=3071MB, multi_processor_count=10)
   Allocated :  0
   Reserved  :  0

| ID | GPU  | MEM |
-------------------
|  0 | nan% |  1% |


In [2]:
pp = pprint.PrettyPrinter(indent=4)
np.set_printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')
torch.set_printoptions(precision=6, linewidth=132)
# torch.set_printoptions(precision=None, threshold=None, edgeitems=None, linewidth=None, profile=None, sci_mode=None)
pd.options.display.width = 132

In [3]:
# sys.path.pop(0)
# sys.path.insert(0, '/home/kbardool/kusanagi/AdaSparseChem/src')
# print(sys.path)

## Read yaml config file

In [4]:
input_args = " --config yamls/adashare/chembl_2task.yaml --cpu --batch_size 09999".split()
# get command line arguments
args = get_command_line_args(input_args)

if args.exp_instance is None:
    args.exp_instance = datetime.now().strftime("%m%d_%H%M")
    
# print(args.exp_instance)
# print(args)
# print()

 command line parms :  {'config': 'yamls/adashare/chembl_2task.yaml', 'exp_instance': None, 'exp_ids': [0], 'batch_size': 9999, 'backbone_lr': None, 'task_lr': None, 'decay_lr_rate': None, 'decay_lr_freq': None, 'gpus': [0], 'cpu': True}


In [5]:
print_separator('READ YAML')

opt, gpu_ids, _ = read_yaml_from_input(args)

fix_random_seed(opt["seed"][1])

opt['exp_description'] = f"No Alternating Weight/Policy - training all done with both weights and policy"
opt['exp_log_dir']        =  os.path.join(opt['paths']['log_dir'], opt['paths']['exp_folder'])
opt['exp_checkpoint_dir'] =  os.path.join(opt['paths']['checkpoint_dir'], opt['paths']['exp_folder'])
# folder_name=  f"{opt['exp_instance']}_bs{opt['train']['batch_size']:03d}_{opt['train']['decay_lr_rate']:3.2f}_{opt['train']['decay_lr_freq']}"

print_heading(f" experiment name       : {opt['exp_name']} \n"
              f" experiment instance   : {opt['exp_instance']} \n"
              f" folder_name           : {opt['paths']['exp_folder']} \n"
              f" experiment description: {opt['exp_description']}\n"
              f" log folder            : {opt['exp_log_dir']}\n"
              f" checkpoint folder     : {opt['exp_checkpoint_dir']}", verbose = True)

##################################################
####################READ YAML#####################
##################################################
------------------------------------------------------------------------------------------------------------------------
 experiment name       : SparseChem 
 experiment instance   : 0215_1001 
 folder_name           : 50x6_0215_1001_plr0.0001_sp1e-06_sh10.0 
 experiment description: No Alternating Weight/Policy - training all done with both weights and policy
 log folder            : ../experiments/AdaSparseChem/50x6_0215_1001_plr0.0001_sp1e-06_sh10.0
 checkpoint folder     : ../experiments/AdaSparseChem/50x6_0215_1001_plr0.0001_sp1e-06_sh10.0
------------------------------------------------------------------------------------------------------------------------ 



In [6]:
# for line in lines: 
create_path(opt)    

# print yaml on the screen
for line in print_yaml2(opt):
    print(line)

write_parms_report(opt)    

 Create folder ../experiments/AdaSparseChem/50x6_0215_1001_plr0.0001_sp1e-06_sh10.0
            exp_name : SparseChem
        exp_instance : 0215_1001
     exp_description : No Alternating Weight/Policy - training all done with both weights and policy
                seed : [88, 45, 50, 100, 44, 48, 2048, 2222, 9999]
            backbone : SparseChem
       backbone_orig : ResNet18
          orig_tasks : ['seg', 'sn']
               tasks : ['class', 'class', 'class']
     tasks_num_class : [5, 5, 5]
             lambdas : [1, 1, 1]
        policy_model : task-specific
             verbose : False
     input_size_freq : None
          input_size : 32000
        hidden_sizes : [50, 50, 50, 50, 50, 50]
    tail_hidden_size : 50
 first_non_linearity : relu
middle_non_linearity : relu
      middle_dropout : 0.2
  last_non_linearity : relu
        last_dropout : 0.2
   class_output_size : None
    regr_output_size : None
              policy : True
           is_sparse : True
          is_s

## Chembl Dataloader V3

In [7]:
trainset  = ClassRegrSparseDataset_v3(opt, split_ratios = opt['dataload']['x_split_ratios'], ratio_index = 0, verbose = False)
trainset1 = ClassRegrSparseDataset_v3(opt, split_ratios = opt['dataload']['x_split_ratios'], ratio_index = 1)
trainset2 = ClassRegrSparseDataset_v3(opt, split_ratios = opt['dataload']['x_split_ratios'], ratio_index = 2)
valset    = ClassRegrSparseDataset_v3(opt, split_ratios = opt['dataload']['x_split_ratios'], ratio_index = 3)

train_loader  = InfiniteDataLoader(trainset , batch_size=opt['train']['batch_size'], num_workers = 2, pin_memory=True, collate_fn=trainset.collate, shuffle=False)
val_loader    = InfiniteDataLoader(valset   , batch_size=opt['train']['batch_size'], num_workers = 1, pin_memory=True, collate_fn=valset.collate  , shuffle=False)
train1_loader = InfiniteDataLoader(trainset1, batch_size=opt['train']['batch_size'], num_workers = 2, pin_memory=True, collate_fn=trainset1.collate, shuffle=False)
train2_loader = InfiniteDataLoader(trainset2, batch_size=opt['train']['batch_size'], num_workers = 2, pin_memory=True, collate_fn=trainset2.collate, shuffle=False)

print(f" trainset.y_class                       :  {[ i.shape  for i in trainset.y_class_list]}")
print(f" trainset1.y_class                      :  {[ i.shape  for i in trainset1.y_class_list]}")
print(f" trainset2.y_class                      :  {[ i.shape  for i in trainset2.y_class_list]}")
print(f" valset.y_class                         :  {[ i.shape  for i in valset.y_class_list  ]} ")
print()
print(f' size of training set 0 (warm up)       :  {len(trainset)}')
print(f' size of training set 1 (network parms) :  {len(trainset1)}')
print(f' size of training set 2 (policy weights):  {len(trainset2)}')
print(f' size of validation set                 :  {len(valset)}')
print(f'                               Total    :  {len(trainset)+len(trainset1)+len(trainset2)+len(valset)}')
print()
print(f" batch size                             :  {opt['train']['batch_size']}")
print()
print(f" # batches training 0 (warm up)         :  {len(train_loader)}")
print(f" # batches training 1 (network parms)   :  {len(train1_loader)}")
print(f" # batches training 2 (policy weights)  :  {len(train2_loader)}")
print(f" # batches validation dataset           :  {len(val_loader)}")
print()

 trainset.y_class                       :  [(13791, 5), (13791, 5), (13791, 5)]
 trainset1.y_class                      :  [(18, 5), (18, 5), (18, 5)]
 trainset2.y_class                      :  [(18, 5), (18, 5), (18, 5)]
 valset.y_class                         :  [(4561, 5), (4561, 5), (4561, 5)] 

 size of training set 0 (warm up)       :  13791
 size of training set 1 (network parms) :  18
 size of training set 2 (policy weights):  18
 size of validation set                 :  4561
                               Total    :  18388

 batch size                             :  128

 # batches training 0 (warm up)         :  108
 # batches training 1 (network parms)   :  1
 # batches training 2 (policy weights)  :  1
 # batches validation dataset           :  36



## Create Environment

### Create model


In [8]:
environ = SparseChemEnv_Dev(log_dir          = opt['exp_log_dir'], 
                            checkpoint_dir   = opt['exp_checkpoint_dir'], 
                            exp_name         = opt['exp_name'],
                            tasks_num_class  = opt['tasks_num_class'], 
                            init_neg_logits  = opt['train']['init_neg_logits'], 
                            device           = gpu_ids[0],
                            init_temperature = opt['train']['init_temp'], 
                            temperature_decay= opt['train']['decay_temp'], 
                            is_train         = True,
                            opt              = opt, 
                            verbose          = False)

cfg = environ.print_configuration()
write_parms_report(opt, cfg, mode = 'a')
environ.networks['mtl-net']

-------------------------------------------------------
* SparseChemEnv_Dev  Initializtion - verbose: False
------------------------------------------------------- 

------------------------------------------------------------
SparseChemEnv_Dev.super() init()  Start - verbose: False
------------------------------------------------------------ 

 log_dir        :  ../experiments/AdaSparseChem/50x6_0215_1001_plr0.0001_sp1e-06_sh10.0 
 checkpoint_dir :  ../experiments/AdaSparseChem/50x6_0215_1001_plr0.0001_sp1e-06_sh10.0 
 exp_name       :  SparseChem 
 tasks_num_class:  [5, 5, 5] 
 device         :  cuda:0 
 device id      :  0 
 dataset        :  Chembl_23_mini 
 tasks          :  ['class', 'class', 'class'] 

--------------------------------------------------
SparseChemEnv_Dev.super() init()  end
-------------------------------------------------- 

 is_train       :  True 
 init_neg_logits:  None 
 init temp      :  4 
 decay temp     :  0.9 
 input_size     :  32000 
 normalize loss :

MTL3_Dev(
  (backbone): SparseChem_Backbone(
    (Input_linear): SparseLinear(in_features=32000, out_features=50, bias=True)
    (blocks): ModuleList(
      (0): ModuleList(
        (0): SparseChemBlock(
          (linear): Linear(in_features=50, out_features=50, bias=True)
          (non_linear): ReLU()
          (dropout): Dropout(p=0.2, inplace=False)
        )
      )
      (1): ModuleList(
        (0): SparseChemBlock(
          (linear): Linear(in_features=50, out_features=50, bias=True)
          (non_linear): ReLU()
          (dropout): Dropout(p=0.2, inplace=False)
        )
      )
      (2): ModuleList(
        (0): SparseChemBlock(
          (linear): Linear(in_features=50, out_features=50, bias=True)
          (non_linear): ReLU()
          (dropout): Dropout(p=0.2, inplace=False)
        )
      )
      (3): ModuleList(
        (0): SparseChemBlock(
          (linear): Linear(in_features=50, out_features=50, bias=True)
          (non_linear): ReLU()
          (dropout): D

## Warmup Training

### Training Preparation

In [9]:
environ.define_optimizer(policy_learning=False)
environ.define_scheduler(policy_learning=False)
# Fix Alpha - 
environ.fix_alpha()
environ.free_weights(opt['fix_BN'])

if opt['train']['resume']:
    print('Resume training')
    current_iter = environ.load(opt['train']['which_iter'])
    environ.networks['mtl-net'].reset_logits()
else:
    print('Initiate Training ')

if torch.cuda.is_available():
    print('cuda available', gpu_ids)   
    environ.cuda(gpu_ids)
else:
    print('cuda not available')
    environ.cpu()
print('\n')

flag           = 'update_w'
current_epoch  = 0
current_iter   = 0
current_iter_w = 0 
current_iter_a = 0
best_value     = 0 
best_iter      = 0
p_epoch        = 0
w_epoch        = 0

best_metrics   = None
flag_warmup    = True
eval_iters     = -1
num_prints     = 0
num_blocks     = sum(environ.networks['mtl-net'].layers)
 
warm_up_epochs     = opt['train']['warm_up_epochs']
train_total_epochs = opt['train']['iteration_epochs']
curriculum_speed   = opt['curriculum_speed'] 

stop_epoch = current_epoch + warm_up_epochs


if opt['train']['print_freq'] == -1:
    print(f" set print_freq to length of train loader: {len(train_loader)}")
    opt['train']['print_freq']    = len(train_loader)

if opt['train']['val_iters'] == -1:
    print(f" set eval_iters to length of val loader  : {len(val_loader)}")
    eval_iters    = len(val_loader)    
else:
    eval_iters    = opt['train']['val_iters']

# opt['train']['weight_iter_alternate'] = opt['train'].get('weight_iter_alternate' , len(train1_loader))
# opt['train']['alpha_iter_alternate']  = opt['train'].get('alpha_iter_alternate'  , len(train2_loader))
opt['train']['weight_iter_alternate'] = len(train_loader)
opt['train']['alpha_iter_alternate']  = len(train_loader)
stop_iter_w = opt['train']['weight_iter_alternate']
stop_iter_a = (opt['train']['alpha_iter_alternate'] )


Initiate Training 
cuda available [0]


 set print_freq to length of train loader: 108
 set eval_iters to length of val loader  : 36


In [10]:
print( 
    f"\n experiment name           : {opt['exp_name']}",
    f"\n experiment description    : {opt['exp_description']}",
    f"                                \n"
    f"\n Network[mtl_net].layers   : {environ.networks['mtl-net'].layers}",
    f"\n Num_blocks                : {sum(environ.networks['mtl-net'].layers)}"    
    f"                                \n"
    f"\n batch size                : {opt['train']['batch_size']}",    
    f"\n Total iterations          : {opt['train']['total_iters']}",
    f"\n Warm-up iterations        : {opt['train']['warm_up_iters']}",
    f"\n Warm-up epochs            : {opt['train']['warm_up_epochs']}",
    f"\n train_total_epochs        : {train_total_epochs}",
    f"                                \n"
    f"\n Print Frequency           : {opt['train']['print_freq']}",
    f"\n Validation Frequency      : {opt['train']['val_freq']}",
    f"\n Validation Iterations     : {opt['train']['val_iters']}",
    f"\n eval_iters                : {eval_iters}",
    f"\n which_iter                : {opt['train']['which_iter']}",
    f"\n train_resume              : {opt['train']['resume']}",
    f"                                \n",                     
    f"\n Length train_loader       : {len(train_loader)}",
    f"\n Length val_loader         : {len(val_loader)}",
    f"\n stop_iter_w               : {stop_iter_w}",
    f"                                \n",
    f"\n fix BN parms              : {opt['fix_BN']}",    
    f"\n Backbone LR               : {opt['train']['backbone_lr']}",
    f"\n Backbone LR               : {opt['train']['task_lr']   }",     
    f"                                \n"
    f"\n Sharing  regularization   : {opt['train']['lambda_sharing']}",    
    f"\n Sparsity regularization   : {opt['train']['lambda_sparsity']}",  
    f"\n Task     regularization   : {opt['train']['lambda_tasks']}")

print(f" Last Epoch: {current_epoch}   # of warm-up epochs to do:  {warm_up_epochs} - Run epochs {current_epoch+1} to {stop_epoch}")


 experiment name           : SparseChem 
 experiment description    : No Alternating Weight/Policy - training all done with both weights and policy                                 

 Network[mtl_net].layers   : [1, 1, 1, 1, 1, 1] 
 Num_blocks                : 6                                

 batch size                : 128 
 Total iterations          : 25000 
 Warm-up iterations        : None 
 Warm-up epochs            : 1 
 train_total_epochs        : 50                                 

 Print Frequency           : 108 
 Validation Frequency      : 500 
 Validation Iterations     : -1 
 eval_iters                : 36 
 which_iter                : warmup 
 train_resume              : False                                 
 
 Length train_loader       : 108 
 Length val_loader         : 36 
 stop_iter_w               : 108                                 
 
 fix BN parms              : False 
 Backbone LR               : 0.01 
 Backbone LR               : 0.01                     

In [11]:
print(
    f"\n folder: {opt['paths']['exp_folder']}",
    f"\n layers: {opt['hidden_sizes']}",    
    f"                               \n",
    f"\n diff_sparsity_weights  : {opt['diff_sparsity_weights']}",
    f"\n skip_layer             : {opt['skip_layer']}",
    f"\n is_curriculum          : {opt['is_curriculum']}",
    f"\n curriculum_speed       : {opt['curriculum_speed']}",
    f"                              \n",    
    f"\n decay_lr_rate          : {opt['train']['decay_lr_rate']}",      
    f"\n decay_lr_freq          : {opt['train']['decay_lr_freq']}",     
    f"\n policy_decay_lr_rate   : {opt['train']['policy_decay_lr_rate']}",      
    f"\n policy_decay_lr_freq   : {opt['train']['policy_decay_lr_freq']}", 
    f"                              \n",    
    f"\n policy_lr              : {opt['train']['policy_lr']}", 
    f"\n lambda_sparsity        : {opt['train']['lambda_sparsity']}",      
    f"\n lambda_sharing         : {opt['train']['lambda_sharing']}", 
    f"                              \n",    
    f"\n lambda_tasks           : {opt['train']['lambda_tasks']}",  
    f"\n init_temp              : {opt['train']['init_temp']}",
    f"\n decay_temp             : {opt['train']['decay_temp']}",    
    f"\n decay_temp_freq        : {opt['train']['decay_temp_freq']}",   
    f"\n init_method            : {opt['train']['init_method']}", 
    f"\n init_neg_logits        : {opt['train']['init_neg_logits']}",    
    f"\n hard_sampling          : {opt['train']['hard_sampling']}",
    f"\n Warm-up epochs         : {opt['train']['warm_up_epochs']}",
    f"\n iteration epochs       : {opt['train']['iteration_epochs']}",
)


 folder: 50x6_0215_1001_plr0.0001_sp1e-06_sh10.0 
 layers: [50, 50, 50, 50, 50, 50]                                
 
 diff_sparsity_weights  : False 
 skip_layer             : 0 
 is_curriculum          : False 
 curriculum_speed       : 1                               
 
 decay_lr_rate          : 0.85 
 decay_lr_freq          : 2000 
 policy_decay_lr_rate   : 0.85 
 policy_decay_lr_freq   : 2200                               
 
 policy_lr              : 0.0001 
 lambda_sparsity        : 1e-06 
 lambda_sharing         : 10.0                               
 
 lambda_tasks           : 1 
 init_temp              : 4 
 decay_temp             : 0.9 
 decay_temp_freq        : 8 
 init_method            : random 
 init_neg_logits        : None 
 hard_sampling          : False 
 Warm-up epochs         : 1 
 iteration epochs       : 50


### Warm-up Training

In [12]:

print(f" Last Epoch: {current_epoch}   # of warm-up epochs to do:  {warm_up_epochs} - Run epochs {current_epoch+1} to {stop_epoch}")

# verbose = False
line_count = 0
t = tqdm(initial = current_epoch, total=stop_epoch, desc=f" Warmup training")

while current_epoch < stop_epoch:
    start_time = time.time()
    current_epoch+=1
    t.update(1)
    current_iter_w  = 0     
    #-----------------------------------------
    # Train & Update the network weights
    #-----------------------------------------   
    with trange(+1, stop_iter_w+1 , initial = 0 , total = stop_iter_w, 
                     position=0, leave= False, desc=f" Warmup Epoch {current_epoch}") as t_warmup :
        for current_iter_w in t_warmup:
            current_iter += 1            

            environ.train()    
            batch = next(train_loader)            
            environ.set_inputs(batch, train_loader.dataset.input_size)
            environ.optimize(opt['lambdas'], 
                             is_policy=False, 
                             flag='update_w', 
                             verbose = False)
        
            t_warmup.set_postfix({'curr_iter':current_iter, 
                                  'Loss': f"{environ.losses['total']['total'].item():.4f}"})
#                                   'row_ids':f"{batch['row_id'][0]}-{batch['row_id'][-1]}"})
        ##--------------------------------------------------------------- 
        ## validation
        ##--------------------------------------------------------------- 
#         if should(current_iter_w, stop_iter_w):
        trn_losses = environ.losses
        environ.print_loss(current_iter, start_time, title = f"[e] Warmup epoch:{current_epoch}    iter:")

        val_metrics = environ.evaluate(val_loader, opt['tasks'], 
                               is_policy       = False, 
                               num_train_layers= None,
                               eval_iters      = eval_iters, 
                               progress        = True,
                               leave           = False,
                               verbose         = False)

        environ.print_metrics(current_iter, start_time, val_metrics, title = f"[v] Warmup epoch:{current_epoch}    iter:")
    
        print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics, line_count, out=[sys.stdout, environ.log_file]) 
        line_count += 1

environ.save_checkpoint('warmup', current_iter)   
environ.display_loss(current_iter, title = f"[e] Warmup epoch:{current_epoch}    iter:")
environ.display_trained_policy(current_epoch,out=[sys.stdout, environ.log_file])
environ.log_file.flush()

 Last Epoch: 0   # of warm-up epochs to do:  1 - Run epochs 1 to 1


 Warmup training:   0%|          | 0/1 [00:00<?, ?it/s]

 Warmup Epoch 1:   0%|          | 0/108 [00:00<?, ?it/s]

/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


validation:   0%|          | 0/36 [00:00<?, ?it/s]

Epoch |  trn loss     trn spar     trn shar  trn total |   logloss   bceloss    aucroc     aucpr |  val loss     val spar     val shar  val total |  time |
1     |   10.0734   2.0795e-06   2.7106e-01    10.3445 |   0.00015   0.68333   0.62691   0.62964 |   10.2477   2.0795e-06   2.7106e-01    10.5188 |  27.9 |
[e] Warmup epoch:1    iter:  108 -  Total Loss: 10.3091     
Task: 10.0381   Sparsity: 2.07947e-06    Sharing: 2.71059e-01 

   1 epochs  softmax        sel        softmax       sel        softmax       sel 
 -----    ---------------   ---     ---------------  ---     ---------------  --- 
   1      0.4999   0.5001    0      0.5004   0.4996   1      0.4994   0.5006   0
   2      0.4998   0.5002    0      0.5001   0.4999   1      0.5004   0.4996   1
   3      0.5002   0.4998    1      0.5002   0.4998   1      0.4997   0.5003   0
   4      0.5002   0.4998    1      0.5004   0.4996   1      0.4991   0.5009   0
   5      0.4999   0.5001    0      0.5001   0.4999   1      0.4997   0.5

## Weight & Policy Training

### Training Preparation

In [13]:
if flag_warmup:
    print_heading( f"** {timestring()} - Training iteration {current_iter}   flag: {flag} \n"
                   f"** Set optimizer and scheduler to policy_learning = True (Switch weight optimizer from ADAM to SGD)\n"
                   f"** Switch from Warm Up training to Alternate training Weights & Policy \n"
                   f"** Take checkpoint and block gradient flow through Policy net", verbose=True)
    environ.define_optimizer(policy_learning=True)
    environ.define_scheduler(policy_learning=True)
    flag_warmup = False
    environ.save_checkpoint('warmup', current_iter)
    environ.fix_alpha()
    flag = 'update_w'
    
# train_total_epochs = 100
# train_total_epochs = 50
print(f"Weight iter alternate  : {opt['train']['weight_iter_alternate'] }")
print(f"Alpha  iter alternate  : {opt['train']['alpha_iter_alternate'] }")
print(f"stop_iter_w (Batches in weight epoch)  : {stop_iter_w}")
print(f"stop_iter_a (Batches in policy epoch)7 : {stop_iter_a}")
print()
print(f"current_iters          : {current_iter}")  
print(f"current_epoch          : {current_epoch}") 
print(f"train_total_epochs     : {train_total_epochs}") 

------------------------------------------------------------------------------------------------------------------------
** 2022-02-15 10:02:22:156772 - Training iteration 108   flag: update_w 
** Set optimizer and scheduler to policy_learning = True (Switch weight optimizer from ADAM to SGD)
** Switch from Warm Up training to Alternate training Weights & Policy 
** Take checkpoint and block gradient flow through Policy net
------------------------------------------------------------------------------------------------------------------------ 

Weight iter alternate  : 108
Alpha  iter alternate  : 108
stop_iter_w (Batches in weight epoch)  : 108
stop_iter_a (Batches in policy epoch)7 : 108

current_iters          : 108
current_epoch          : 1
train_total_epochs     : 50


### Weight/Policy Training

In [ ]:
print_heading(f" Last Epoch: {current_epoch}   # of iteration epochs to do:  {train_total_epochs} - Run epochs {current_epoch+1} to {train_total_epochs}", verbose = True)
leave      = False
verbose    = False
line_count = 0
t = tqdm(initial = current_epoch, total=train_total_epochs, desc=f" Alternate Weight/Policy training")

while current_epoch < train_total_epochs:
    current_epoch+=1
    t.update(1)
    #-----------------------------------------------------------------------------------------------------------
    # Set number of layers to train based on cirriculum_speed and p_epoch (number of epochs of policy training)
    # e.g., When curriculum_speed == 3, num_train_layers is incremented  after every 3 policy training epochs
    #-----------------------------------------------------------------------------------------------------------
    num_train_layers = (p_epoch // opt['curriculum_speed']) + 1  if opt['is_curriculum'] else None

    #-----------------------------------------
    # Train & Update the network weights
    #-----------------------------------------
    if flag == 'update_w':
        start_time = time.time()

        with trange(+1, stop_iter_w+1 , initial = 0, total = stop_iter_w, position=0,
                     leave= leave, desc=f"Epoch {current_epoch} weight training") as t_weights :
            
            for current_iter_w in t_weights:    
                current_iter += 1
                environ.train()
                batch = next(train_loader)

                environ.set_inputs(batch, train1_loader.dataset.input_size)
 
                environ.optimize(opt['lambdas'], 
                                 is_policy=opt['policy'], 
                                 flag=flag, 
                                 num_train_layers=num_train_layers,
                                 hard_sampling=opt['train']['hard_sampling'],
                                 verbose = False)

                t_weights.set_postfix({'iter': current_iter, 
                                       'Loss': f"{environ.losses['losses']['total'].item():.4f}" , 
                                       'Spar': f"{environ.losses['sparsity']['total'].item():.4e}",  
                                       'Shar': f"{environ.losses['sharing']['total'].item():.4e}"})  

        #--------------------------------------------------------------------
        # validation process (here current_iter_w and stop_iter_w are equal)
        #--------------------------------------------------------------------
        trn_losses = environ.losses
        environ.print_loss(current_iter, start_time, title = f"[e] Weight training epoch:{current_epoch}    iter:", to_display = False)

        val_metrics = environ.evaluate(val_loader, opt['tasks'], is_policy=opt['policy'],
                                       num_train_layers=num_train_layers, hard_sampling=opt['train']['hard_sampling'],
                                       eval_iters = eval_iters, progress = True, leave = leave, verbose = False)  

        environ.print_metrics(current_iter, start_time, val_metrics, title = f"[v] Weight training epoch:{current_epoch}    iter:", verbose = False)
        print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics, line_count, out=[sys.stdout, environ.log_file]) 
        line_count += 1

        # Take check point:     environ.save_checkpoint('latest_weights', current_iter)
        #------------------------------------------------------------------------ 
        #  Save Best Checkpoint Code (saved below and in sparsechem_env_dev.py)
        #----------------------------------------------------------------------- 

        #-----------------------------------------------------------------------
        # END validation process - prepare for policy training iteration
        #-----------------------------------------------------------------------
        flag = 'update_alpha'
        environ.fix_weights()
        environ.free_alpha()

    #-----------------------------------------
    # Policy Training  
    #-----------------------------------------
    if flag == 'update_alpha':
        start_time = time.time()        

        with trange( +1, stop_iter_a+1 , initial = 0, total = stop_iter_a,  position=0,
                     leave= leave, desc=f"Epoch {current_epoch} policy training") as t_policy :
            for current_iter_a in t_policy:    
                current_iter += 1
                batch = next(train_loader)

                environ.set_inputs(batch, train2_loader.dataset.input_size)

                environ.optimize(opt['lambdas'], is_policy=opt['policy'], 
                                 flag=flag, num_train_layers=num_train_layers,
                                 hard_sampling=opt['train']['hard_sampling'], verbose = False)
                
                t_policy.set_postfix({'iter': current_iter, 
                                      'Loss': f"{environ.losses['losses']['total'].item():.4f}" , 
                                      'Spar': f"{environ.losses['sparsity']['total'].item():.4e}" ,  
                                      'Shar': f"{environ.losses['sharing']['total'].item():.4e}",
                                      'lyrs': f"{num_train_layers}"})    
#                                       'row_ids':f"{batch['row_id'][0]}-{batch['row_id'][-1]}"})
        #---------------------------------------------------------------------
        # print loss results (here current_iter_w and stop_iter_w are equal)
        #---------------------------------------------------------------------
        trn_losses = environ.losses
        environ.print_loss(current_iter, start_time, title = f"[e] Policy training epoch:{current_epoch}    iter:")

        val_metrics = environ.evaluate(val_loader, opt['tasks'], is_policy=opt['policy'],
                                       num_train_layers=num_train_layers, hard_sampling=opt['train']['hard_sampling'],
                                       eval_iters = eval_iters, progress = True, leave = False, verbose = False)  

        environ.print_metrics(current_iter, start_time, val_metrics, title = f"[v] Policy training epoch:{current_epoch}    iter:", verbose = False)
        print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics, line_count, out=[sys.stdout, environ.log_file])      
        line_count += 1
        
        p_epoch += 1        
        if should(p_epoch, opt['train']['decay_temp_freq']):
            environ.decay_temperature()
            print(f" decay gumbel softmax to {environ.temp}")
        
        flag = 'update_w'
        environ.fix_alpha()
        environ.free_weights(opt['fix_BN'])

    if should(current_epoch, 10):
        environ.log_file.flush()
        environ.save_checkpoint('latest_weights_policy', current_iter)        
        environ.display_trained_policy(current_epoch,out=[sys.stdout, environ.log_file])

environ.save_checkpoint('final_weights_policy', current_iter)   
environ.display_loss(current_iter, title = f"[e] Policy training epoch:{current_epoch}    iter:", to_display = True)
environ.display_trained_policy(current_epoch,out=[sys.stdout, environ.log_file])
environ.log_file.flush()

--------------------------------------------------------------------------
 Last Epoch: 1   # of iteration epochs to do:  50 - Run epochs 2 to 50
-------------------------------------------------------------------------- 



 Alternate Weight/Policy training:   2%|2         | 1/50 [00:00<?, ?it/s]

Epoch 2 weight training:   0%|          | 0/108 [00:00<?, ?it/s]

/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


validation:   0%|          | 0/36 [00:00<?, ?it/s]

...

Epoch 22 policy training:   0%|          | 0/108 [00:00<?, ?it/s]

validation:   0%|          | 0/36 [00:00<?, ?it/s]

22    |    4.9413   2.0848e-06   2.6675e-03     4.9440 |   0.00017   0.75618   0.75532   0.75769 |   11.3252   2.0848e-06   2.4624e-03    11.3276 |  35.5 |


Epoch 23 weight training:   0%|          | 0/108 [00:00<?, ?it/s]

validation:   0%|          | 0/36 [00:00<?, ?it/s]

In [ ]:
print( f"\n Sparsity regularization   : {opt['train']['lambda_sparsity']}",
       f"\n Sharing  regularization   : {opt['train']['lambda_sharing']}",
       f"\n current_epochs               : {current_epoch}") # 

In [32]:
# environ.print_loss(current_iter, start_time, title = f"[e]Policy training epoch:{current_epoch}   iteration:", verbose=True)
# environ.display_trained_policy(current_epoch, out = [None, environ.log_file])
# environ.log_file.flush()

In [21]:
train_total_epochs += 3
     
print(f"current_iters         : {current_iter}")  
print(f"current_epochs           : {current_epoch}") 
print(f"train_total_epochs    : {train_total_epochs}") 

current_iters         : 540
curr_epochs           : 3
train_total_epochs    : 6


In [20]:
pp.pprint(environ.losses)

{   'losses': {   'task1': tensor(3.384938, device='cuda:0', dtype=torch.float64),
                  'task2': tensor(3.544476, device='cuda:0', dtype=torch.float64),
                  'task3': tensor(4.924039, device='cuda:0', dtype=torch.float64),
                  'total': tensor(11.853453, device='cuda:0', dtype=torch.float64)},
    'losses_mean': {   'task1': tensor(0.676988, device='cuda:0', dtype=torch.float64),
                       'task2': tensor(0.708895, device='cuda:0', dtype=torch.float64),
                       'task3': tensor(0.984808, device='cuda:0', dtype=torch.float64),
                       'total': tensor(2.370691, device='cuda:0', dtype=torch.float64)},
    'parms': {   'gumbel_temp': 4,
                 'lr_0': 0.01,
                 'lr_1': 0.01,
                 'policy_lr': 0.0001,
                 'train_layers': 0},
    'sharing': {'total': tensor(0.000328, device='cuda:0')},
    'sparsity': {   'task1_logits': tensor(6.913636e-08, device='cuda:0'),
     

## Post Training Stuff

### Post Warm-up Training stuff

In [15]:
pp.pprint(environ.val_metrics)

{   'aggregated': {   'auc_pr': 0.5683542231355692,
                      'avg_prec_score': 0.5686955149511226,
                      'bceloss': 0.6940069357554117,
                      'f1_max': 0.6753742658177068,
                      'kappa': 0.08318090860174578,
                      'kappa_max': 0.11385237142207795,
                      'logloss': tensor(0.0002, device='cuda:0', dtype=torch.float64),
                      'p_f1_max': 0.29707588851451877,
                      'p_kappa_max': 0.48534467220306404,
                      'roc_auc_score': 0.5679242272843051,
                      'sc_loss': tensor(0.2891, device='cuda:0', dtype=torch.float64)},
    'epoch': 1,
    'loss': {   'task1': 3.5098743182605396,
                'task2': 3.4508372449718867,
                'task3': 3.4480013587242775,
                'total': 10.408712921956704},
    'loss_mean': {   'task1': 0.701974863652108,
                     'task2': 0.6901674489943774,
                     'task3': 0.

In [16]:
environ.networks['mtl-net'].arch_parameters()

[Parameter containing:
 tensor([[0.5000, 0.5000],
         [0.5000, 0.5000],
         [0.5000, 0.5000]], device='cuda:0'),
 Parameter containing:
 tensor([[0.5000, 0.5000],
         [0.5000, 0.5000],
         [0.5000, 0.5000]], device='cuda:0'),
 Parameter containing:
 tensor([[0.5000, 0.5000],
         [0.5000, 0.5000],
         [0.5000, 0.5000]], device='cuda:0')]

In [31]:
p = environ.get_sample_policy(hard_sampling = False)
print(p)
p = environ.get_policy_prob()
print(p)
p = environ.get_policy_logits()
print(p)

# p = environ.get_current_policy()
# print(p)

([tensor([[1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1]], device='cuda:0'), tensor([[0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1]], device='cuda:0'), tensor([[1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1]], device='cuda:0')], [array([[0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5]], dtype=float32), array([[0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5]], dtype=float32), array([[0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5]], dtype=float32)])
[array([[0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5]], dtype=float32), array([[0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5]], dtype=floa

In [42]:
a = softmax([0.0, 1])
print(a)
sampled = np.random.choice((1, 0), p=a)
print(sampled)

[0.26894142 0.73105858]
1


In [20]:
print(environ.optimizers['weights'])
print(environ.schedulers['weights'].get_last_lr())

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.5, 0.999)
    eps: 1e-08
    initial_lr: 0.01
    lr: 0.01
    weight_decay: 0.0001

Parameter Group 1
    amsgrad: False
    betas: (0.5, 0.999)
    eps: 1e-08
    initial_lr: 0.01
    lr: 0.01
    weight_decay: 0.0001
)
[0.01, 0.01]


In [17]:
print('losses.keys      : ', environ.losses.keys())
print('losses[task]keys : ', environ.losses['task1'].keys())
pp.pprint(environ.losses)

losses.keys      :  dict_keys(['parms', 'losses', 'losses_mean', 'sparsity', 'sharing', 'total', 'total_mean', 'task1', 'task2', 'task3'])
losses[task]keys :  dict_keys(['cls_loss', 'cls_loss_mean'])
{   'losses': {   'task1': tensor(3.5704, device='cuda:0', dtype=torch.float64),
                  'task2': tensor(3.3313, device='cuda:0', dtype=torch.float64),
                  'task3': tensor(3.3721, device='cuda:0', dtype=torch.float64),
                  'total': tensor(10.2737, device='cuda:0', dtype=torch.float64)},
    'losses_mean': {   'task1': tensor(0.7141, device='cuda:0', dtype=torch.float64),
                       'task2': tensor(0.6663, device='cuda:0', dtype=torch.float64),
                       'task3': tensor(0.6744, device='cuda:0', dtype=torch.float64),
                       'total': tensor(2.0547, device='cuda:0', dtype=torch.float64)},
    'parms': {'gumbel_temp': 5, 'lr_0': 0.01, 'lr_1': 0.01, 'train_layers': 0},
    'sharing': {'total': tensor(0., device='cuda:

In [20]:
print( environ.val_metrics.keys())
# pp.pprint(val_metrics)
print(type(environ.val_metrics['aggregated']))
print()
print(type(environ.val_metrics['task1']['classification_agg']))
print()
pp.pprint(environ.val_metrics)

dict_keys(['loss', 'loss_mean', 'task1', 'task2', 'task3', 'aggregated', 'train_time', 'epoch'])
<class 'dict'>

<class 'dict'>

{   'aggregated': {   'auc_pr': 0.8163426647572675,
                      'avg_prec_score': 0.8164138615123665,
                      'bceloss': 0.7729340473810831,
                      'f1_max': 0.7574111413660235,
                      'kappa': 0.4593877004350045,
                      'kappa_max': 0.4722816344851592,
                      'logloss': tensor(0.0002, device='cuda:0', dtype=torch.float64),
                      'p_f1_max': 0.1918067594369252,
                      'p_kappa_max': 0.545021508137385,
                      'roc_auc_score': 0.8101998985724014,
                      'sc_loss': tensor(0.3217, device='cuda:0', dtype=torch.float64)},
    'epoch': 4000,
    'loss': {   'task1': 3.990690022259455,
                'task2': 3.7726694706664947,
                'task3': 3.817710672775008,
                'total': 11.581070165700957},
    'l

In [20]:
# import pickle
# with open("val_metrics.pkl", mode= 'wb') as f:
#         pickle.dump(val_metrics, f)
    
# with open('val_metrics.pkl', 'rb') as f:    
#     tst_val_metrics = pickle.load(f)

In [21]:
# print(environ.input.shape) 
# a = getattr(environ, 'task1_pred')
# yc_data = environ.batch['task1_data']
# print(yc_data.shape)
# yc_ind = environ.batch['task1_ind']
# print(yc_ind.shape)
# yc_hat_all = getattr(environ, 'task1_pred')
# print(yc_hat_all.shape)
# yc_hat  = yc_hat_all[yc_ind[0], yc_ind[1]]
# print(yc_hat_all.shape, yc_hat.shape)

# 
# environ.losses
# loss = {}
# for key in environ.losses.keys():
#     loss[key] = {}
#     for subkey, v in environ.losses[key].items():
#         print(f" key:  {key}   subkey: {subkey} ")
#         if isinstance(v, torch.Tensor):
#             loss[key][subkey] = v.data
#             print(f" Tensor  -  key:  {key}   subkey: {subkey}           value type: {type(v)}  value: {v:.4f}")
#         else:
#             loss[key][subkey] = v
#             print(f" integer -  key:  {key}   subkey: {subkey}           value type: {type(v)}  value: {v:.4f}")
# pp.pprint(tst_val_metrics)             

In [22]:
# print('metrics.keys: ', environ.metrics.keys())
# print('metrics[task].keys: ', environ.metrics['task1'].keys())
# pp.pprint(environ.metrics['task1'])
# pp.pprint(environ.losses['task1']['total'])

In [23]:
# title='Iteration'
# for t_id, _ in enumerate(environ.tasks):
#     task_key = f"task{t_id+1}"
# #     print_heading(f"{title}  {current_iter}  {task_key} : {val_metrics[task_key]['classification_agg']}", verbose = True)

#     for key, _  in val_metrics[task_key]['classification_agg'].items():
#         print('%s/%-20s'%(task_key, key), val_metrics[task_key]['classification_agg'][key], current_iter)
#         print(f"{task_key:s}/{key:20s}", val_metrics[task_key]['classification_agg'][key], current_iter)
#         print()
#             # print_current_errors(os.path.join(self.log_dir, 'loss.txt'), current_iter,key, loss[key], time.time() - start_time)

In [32]:
# environ.print_loss(current_iter, start_time, metrics = val_metrics['loss'], verbose=True)
# print(opt['lambdas'])
# p = (opt['lambdas'][0] * environ.losses['tasks']['task1'])
# print(p)

# environ.print_metrics(current_iter, start_time, val_metrics , title='validation', verbose=True)    

In [30]:
# print(current_iter)
# print_metrics_cr(current_iter, t1 - t0, None, val_metrics , True)
# environ.print_metrics(current_iter, start_time, val_metrics, title='validation', verbose = True)

In [21]:
print(f" val_metric keys               : {val_metrics.keys()}")
print(f" loss keys                     : {val_metrics['loss'].keys()}")
print(f" task1 keys                    : {val_metrics['task1'].keys()}")
print(f" task1 classification keys     : {val_metrics['task1']['classification'].keys()}")
print(f" task1 classification_agg keys : {val_metrics['task1']['classification_agg'].keys()}")
print()
print(f" task1                       : {val_metrics['task1']['classification_agg']['loss']:5f}")
print(f" task2                       : {val_metrics['task2']['classification_agg']['loss']:5f}")
print(f" task3                       : {val_metrics['task3']['classification_agg']['loss']:5f}")
print(f" loss                        : {val_metrics['loss']['total']:5f}")
print(f" train_time                  : {val_metrics['train_time']:2f}")
print(f" epoch                       : {val_metrics['epoch']}")


AttributeError: 'NoneType' object has no attribute 'keys'

### Post Weight + Policy Training Stuff 

In [178]:
environ.networks['mtl-net'].backbone.layer_config

[1, 1, 1, 1, 1, 1]

In [63]:
num_blocks = 6
num_policy_layers = 6
gt =  torch.ones((num_blocks)).long()
gt0 =  torch.zeros((num_blocks)).long()
print(gt)
print(gt0)

loss_weights = ((torch.arange(0, num_policy_layers, 1) + 1).float() / num_policy_layers)
print(loss_weights)

tensor([1, 1, 1, 1, 1, 1])
tensor([0, 0, 0, 0, 0, 0])
tensor([0.1667, 0.3333, 0.5000, 0.6667, 0.8333, 1.0000])


In [77]:
if environ.opt['diff_sparsity_weights'] and not environ.opt['is_sharing']:
    print(' cond 1')
    ## Assign higher weights to higher layers 
    loss_weights = ((torch.arange(0, num_policy_layers, 1) + 1).float() / num_policy_layers)
    print(f"{task_key} sparsity error:  {2 * (loss_weights[-num_blocks:] * environ.cross_entropy2(logits[-num_blocks:], gt)).mean()})")
    print_dbg(f" loss_weights :  {loss_weights}", verbose = True)
    print_dbg(f" cross_entropy:  {environ.cross_entropy2(logits[-num_blocks:], gt)}  ", verbose = True)
    print_dbg(f" loss[sparsity][{task_key}]: {self.losses['sparsity'][task_key] } ", verbose = True)

else:
    print('\n cond 2')
    print_dbg(f"Compute CrossEntropyLoss between \n Logits   : \n{logits[-num_blocks:]} \n and gt: \n{gt} \n", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-num_blocks:], gt)}")
    
    print('\n cond 2')
    print_dbg(f"Compute CrossEntropyLoss between Logits      : {logits[-1:]}  and gt: {gt[-1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-1:], gt[-1:])} \n")
    print_dbg(f"Compute CrossEntropyLoss between Logits      : {logits[-1:]}  and gt: {gt0[-1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-1:], gt0[-1:])} \n")
    
    print('\n cond 3')    
    print_dbg(f"Compute CrossEntropyLoss between Logits   : {logits[0:1]}  and gt: {gt[0:1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[0:1], gt[0:1])} \n")
    print_dbg(f"Compute CrossEntropyLoss between Logits   : {logits[0:1]}  and gt: {gt0[0:1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[0:1], gt0[0:1])} \n")
        
        

cond 2
Compute CrossEntropyLoss between 
 Logits   : 
tensor([[0.3306, 0.4518],
        [0.3532, 0.5529],
        [0.3888, 0.6125],
        [0.4204, 0.7685],
        [0.4520, 0.7994],
        [0.4840, 0.8021]]) 
 and gt: 
tensor([1, 1, 1, 1, 1, 1]) 

task1_logits sparsity error:  0.5725929141044617

 cond 2
Compute CrossEntropyLoss between Logits      : tensor([[0.4840, 0.8021]])  and gt: 1 
task1_logits sparsity error:  0.5467103123664856 

Compute CrossEntropyLoss between Logits      : tensor([[0.4840, 0.8021]])  and gt: 0 
task1_logits sparsity error:  0.864768385887146 


 cond 3
Compute CrossEntropyLoss between Logits   : tensor([[0.3306, 0.4518]])  and gt: tensor([1]) 
task1_logits sparsity error:  0.634384036064148 

Compute CrossEntropyLoss between Logits   : tensor([[0.3306, 0.4518]])  and gt: tensor([0]) 
task1_logits sparsity error:  0.7555801868438721 



In [24]:
# flag = 'update_w'
# environ.fix_alpha
# environ.free_w(opt['fix_BN'])

flag = 'update_alpha'
environ.fix_weights()
environ.free_alpha()

In [29]:
environ.networks['mtl-net'].num_layers

6

In [25]:
print(f"current_iters         : {current_iter}")  
print(f"current_epochs           : {current_epoch}") 
print(f"train_total_epochs    : {train_total_epochs}") 

train_total_epochs += 5

print(f"current_iters         : {current_iter}")  
print(f"current_epochs           : {current_epoch}") 
print(f"train_total_epochs    : {train_total_epochs}") 

current_iters         : 6580
curr_epochs           : 60
train_total_epochs    : 60


In [25]:
# print_metrics_cr(current_epoch, time.time() - t0, None, environ.val_metrics , num_prints)      

# num_prints += 1
# t0 = time.time()

# # Take check point
# environ.save_checkpoint('latest', current_iter)
# environ.train()
# #-------------------------------------------------------
# # END validation process
# #-------------------------------------------------------       
# flag = 'update_alpha'
# environ.fix_w()
# environ.free_alpha()

Epoch | logloss bceloss  aucroc   aucpr  f1_max| t1 loss t2 loss t3 lossttl loss|tr_time|
1     | 0.00020 0.89942 0.81016 0.81627 0.75738|  4.6591  4.3649  4.4500 13.4740| 333.4|

In [71]:
# dilation = 2
# kernel_size = np.asarray((3, 3))
# upsampled_kernel_size = (kernel_size - 1) * (dilation - 1) + kernel_size
# print(upsampled_kernel_size)

In [30]:
# environ.optimizers['weights'].param_groups[0]
# for param_group in optimizer.param_groups:
#     return param_group['lr']

In [31]:
environ.schedulers['weights'].get_last_lr()

[1e-05, 1e-05]

In [86]:
current_state = {}
for k, v in environ.optimizers.items():
    print(f'state dict for {k} = {v}')
    current_state[k] = v.state_dict()
pp.pprint(current_state)

state dict for weights = SGD (
Parameter Group 0
    dampening: 0
    lr: 0.0001
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0001

Parameter Group 1
    dampening: 0
    lr: 0.0001
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0001
)
state dict for alphas = Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 0.0005
)
{   'alphas': {   'param_groups': [   {   'amsgrad': False,
                                          'betas': (0.9, 0.999),
                                          'eps': 1e-08,
                                          'lr': 0.0001,
                                          'params': [0, 1, 2],
                                          'weight_decay': 0.0005}],
                  'state': {   0: {   'exp_avg': tensor([[ 0.0607, -0.0007],
        [-0.0428, -0.0069],
        [-0.1218,  0.0138],
        [ 0.0086,  0.0238]], device='cuda:0'),
                                      'exp_

In [88]:
current_state = {}
for k, v in environ.schedulers.items():
    print(f'state dict for {k} = {v}')
    print(v.state_dict())

state dict for weights = <torch.optim.lr_scheduler.StepLR object at 0x7f90c01c0ca0>
{'step_size': 4000, 'gamma': 0.5, 'base_lrs': [0.0001, 0.0001], 'last_epoch': 9100, '_step_count': 9101, 'verbose': False, '_get_lr_called_within_step': False, '_last_lr': [2.5e-05, 2.5e-05]}


### Losses and Metrics

In [27]:
trn_losses = environ.losses

In [39]:
print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics , num_prints)      

Epoch |  trn loss     trn spar     trn shar    trn total |   logloss   bceloss    aucroc     aucpr |  val loss     val spar     val shar    val total |tr_time |
12    |    7.3148   1.0717e+00   2.6055e-03       8.3891 |   0.00021   0.97756   0.68488   0.68055 |   14.6682   1.0160e+00   2.1285e-03      15.6863 |  227.5 |

In [27]:
# print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics , num_prints)      

In [26]:
# pp.pprint(environ.losses)
pp.pprint(trn_losses)

{   'losses': {   'task1': tensor(2.450312, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>),
                  'task2': tensor(2.467013, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>),
                  'task3': tensor(2.687793, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>),
                  'total': tensor(7.605118, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)},
    'losses_mean': {   'task1': tensor(0.490062, device='cuda:0', dtype=torch.float64, grad_fn=<MulBackward0>),
                       'task2': tensor(0.493403, device='cuda:0', dtype=torch.float64, grad_fn=<MulBackward0>),
                       'task3': tensor(0.537559, device='cuda:0', dtype=torch.float64, grad_fn=<MulBackward0>),
                       'total': tensor(1.521024, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)},
    'parms': {   'gumbel_temp': 2.6244000000000005,
                 'lr_0': 0.01,
                 'lr_1': 0.01,
        

In [38]:
pp.pprint(environ.val_metrics)

{   'aggregated': {   'auc_pr': 0.6805474562906618,
                      'avg_prec_score': 0.6807816533341606,
                      'bceloss': 0.9775612473487854,
                      'f1_max': 0.6969378498338702,
                      'kappa': 0.2500278499434187,
                      'kappa_max': 0.29669160426755137,
                      'logloss': tensor(0.000214, device='cuda:0', dtype=torch.float64),
                      'p_f1_max': 0.2160851760388027,
                      'p_kappa_max': 0.5119348396857579,
                      'roc_auc_score': 0.6848840436108541,
                      'sc_loss': tensor(0.407449, device='cuda:0', dtype=torch.float64)},
    'epoch': 12,
    'loss': {   'task1': 3.91353933321791,
                'task2': 4.751992322450581,
                'task3': 6.002632442335283,
                'total': 14.668164098003771},
    'loss_mean': {   'task1': 0.782707866643582,
                     'task2': 0.9503984644901166,
                     'task3': 1.20

In [187]:
# environ.opt['train']['Lambda_sharing'] = 0.5
# opt['train']['Lambda_sharing'] = 0.5

# environ.opt['train']['policy_lr'] = 0.001
# opt['train']['policy_lr'] = 0.001

In [38]:
environ.losses.keys()
pp.pprint(environ.losses)

{   'losses': {   'task1': tensor(2.3789, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>),
                  'task2': tensor(2.1948, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>),
                  'task3': tensor(2.3476, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>),
                  'total': tensor(6.9212, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)},
    'losses_mean': {   'task1': tensor(0.4758, device='cuda:0', dtype=torch.float64, grad_fn=<MulBackward0>),
                       'task2': tensor(0.4390, device='cuda:0', dtype=torch.float64, grad_fn=<MulBackward0>),
                       'task3': tensor(0.4695, device='cuda:0', dtype=torch.float64, grad_fn=<MulBackward0>),
                       'total': tensor(1.3842, device='cuda:0', dtype=torch.float64, grad_fn=<AddBackward0>)},
    'parms': {'gumbel_temp': 2.6330653896376153, 'lr_0': 0.005, 'lr_1': 0.005},
    'sharing': {   'total': tensor(2.1338e-06, device='cud

In [79]:
tmp = environ.get_loss_dict()
print(tmp.keys())
pp.pprint(tmp)

dict_keys(['task1', 'task2', 'task3', 'tasks', 'total', 'sharing', 'sparsity'])
{   'sharing': {'total': tensor(0.0002, device='cuda:0')},
    'sparsity': {   'task1_logits': tensor(0.5147, device='cuda:0'),
                    'task2_logits': tensor(0.5195, device='cuda:0'),
                    'task3_logits': tensor(0.4978, device='cuda:0'),
                    'total': tensor(0.0766, device='cuda:0')},
    'task1': {'total': tensor(3.3657, device='cuda:0', dtype=torch.float64)},
    'task2': {'total': tensor(3.5906, device='cuda:0', dtype=torch.float64)},
    'task3': {'total': tensor(3.2182, device='cuda:0', dtype=torch.float64)},
    'tasks': {   'task1': tensor(3.3657, device='cuda:0', dtype=torch.float64),
                 'task2': tensor(3.5906, device='cuda:0', dtype=torch.float64),
                 'task3': tensor(3.2182, device='cuda:0', dtype=torch.float64),
                 'total': tensor(10.1745, device='cuda:0', dtype=torch.float64)},
    'total': {'total': tensor(10.25

In [188]:
print(opt['diff_sparsity_weights'])
print(opt['is_sharing'])
print(opt['diff_sparsity_weights'] and not opt['is_sharing'])
print(environ.opt['train']['Lambda_sharing'])
print(opt['train']['Lambda_sharing'])
print(environ.opt['train']['Lambda_sparsity'])
print(opt['train']['Lambda_sparsity'])
print(environ.opt['train']['policy_lr'])
print(opt['train']['policy_lr'])

True
True
False
0.5
0.5
0.05
0.05
0.001
0.001


### Policy / Logit stuff

In [93]:
from scipy.special          import softmax

In [33]:
np.set_printoptions(precision=8,edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')
torch.set_printoptions(precision=8,linewidth=132)

#### `get_task_logits(n)` Get logits for task group n

In [128]:
task_logits = environ.get_task_logits(1)
print(task_logits)

Parameter containing:
tensor([[-0.00035114, -0.06397165],
        [ 0.00056738, -0.03663344],
        [ 0.00056098, -0.02617791],
        [-0.00044851, -0.07137010],
        [ 0.00013184, -0.05879313],
        [ 0.00079021, -0.05743587]], device='cuda:0')


#### `get_arch_parameters()`: Get last used logits from network

In [34]:
import torch.optim as optim
arch_parameters      = environ.get_arch_parameters()
print(arch_parameters)

[Parameter containing:
tensor([[-0.00035120, -0.06617914],
        [ 0.00056736, -0.04341661],
        [ 0.00056091, -0.01096974],
        [-0.00044879, -0.01083876],
        [ 0.00013163,  0.00874004],
        [ 0.00079006, -0.00861552]], device='cuda:0'), Parameter containing:
tensor([[-0.00035114, -0.06397165],
        [ 0.00056738, -0.03663344],
        [ 0.00056098, -0.02617791],
        [-0.00044851, -0.07137010],
        [ 0.00013184, -0.05879313],
        [ 0.00079021, -0.05743587]], device='cuda:0'), Parameter containing:
tensor([[-0.00035016, -0.06321616],
        [ 0.00056696, -0.03072025],
        [ 0.00056129, -0.01022454],
        [-0.00044983, -0.00021709],
        [ 0.00013071,  0.00484093],
        [ 0.00078938, -0.02230957]], device='cuda:0')]


In [25]:
import torch.optim as optim
arch_parameters      = environ.get_arch_parameters()
print(arch_parameters)

[Parameter containing:
tensor([[ 1.873275e-03, -5.276022e-01],
        [ 2.345233e-03, -2.740704e-01],
        [ 2.614364e-03,  1.604760e-02],
        [ 2.143114e-04,  2.198091e-02],
        [ 4.191113e-04,  5.969038e-02],
        [ 2.007700e-03,  3.544179e-02]], device='cuda:0'), Parameter containing:
tensor([[ 1.873281e-03, -4.892288e-01],
        [ 2.345207e-03, -2.255457e-01],
        [ 2.614349e-03, -2.191145e-01],
        [ 2.143144e-04, -3.354620e-01],
        [ 4.190930e-04, -3.310193e-01],
        [ 2.007697e-03, -2.532191e-01]], device='cuda:0'), Parameter containing:
tensor([[ 1.873283e-03, -6.248206e-01],
        [ 2.345208e-03, -2.149665e-01],
        [ 2.614360e-03, -1.423603e-01],
        [ 2.143196e-04, -1.089546e-01],
        [ 4.191188e-04, -7.532501e-02],
        [ 2.007698e-03, -1.407905e-01]], device='cuda:0')]


#### `get_policy_logits()`:  Get Policy Logits - returns same as `get_arch_parameters()`

In [26]:
logs = environ.get_policy_logits()
for i in logs:
    print(i, '\n')
# probs = softmax(logs, axis= -1)
# for i in probs:
#     print(i, '\n')

[[ 1.8732749e-03 -5.2760220e-01]
 [ 2.3452332e-03 -2.7407044e-01]
 [ 2.6143640e-03  1.6047601e-02]
 [ 2.1431143e-04  2.1980910e-02]
 [ 4.1911125e-04  5.9690382e-02]
 [ 2.0077003e-03  3.5441790e-02]] 

[[ 1.8732806e-03 -4.8922884e-01]
 [ 2.3452067e-03 -2.2554573e-01]
 [ 2.6143489e-03 -2.1911447e-01]
 [ 2.1431442e-04 -3.3546203e-01]
 [ 4.1909298e-04 -3.3101928e-01]
 [ 2.0076970e-03 -2.5321913e-01]] 

[[ 1.8732828e-03 -6.2482059e-01]
 [ 2.3452076e-03 -2.1496648e-01]
 [ 2.6143598e-03 -1.4236034e-01]
 [ 2.1431963e-04 -1.0895463e-01]
 [ 4.1911882e-04 -7.5325012e-02]
 [ 2.0076977e-03 -1.4079048e-01]] 



#### `get_policy_prob()` : Gets the softmax of the logits

In [27]:
policy_softmaxs = environ.get_policy_prob()
for i in policy_softmaxs:
    print(i, '\n')

[[0.6293608  0.37063923]
 [0.5686673  0.43133274]
 [0.49664173 0.5033582 ]
 [0.4945586  0.5054415 ]
 [0.48518652 0.5148135 ]
 [0.49164233 0.50835776]] 

[[0.62036604 0.379634  ]
 [0.55672747 0.44327256]
 [0.5552062  0.44479376]
 [0.58313996 0.4168601 ]
 [0.58210933 0.41789067]
 [0.5634626  0.4365374 ]] 

[[0.6517394  0.34826055]
 [0.5541151  0.44588488]
 [0.5361803  0.46381968]
 [0.5272652  0.47273484]
 [0.518927   0.48107296]
 [0.535639   0.464361  ]] 



#### `get_sample_policy( hard_sampling = False)` : Calls test_sample_policy of network with random choices based on softmax of logits

In [382]:
policy_softmaxs = environ.get_policy_prob()
policies,logits = environ.get_sample_policy(hard_sampling = False)

for l, p, s in zip(logits, policies, policy_softmaxs) :
    for  l_row, p_row, s_row in zip(l, p, s):
        print( l_row,'\t', p_row, '\t', s_row)
    print('\n')

[-0.00035120 -0.06617914] 	 [0 1] 	 [0.51645106 0.48354897]
[ 0.00056736 -0.04341661] 	 [0 1] 	 [0.51099426 0.48900577]
[ 0.00056091 -0.01096974] 	 [1 0] 	 [0.50288266 0.49711737]
[-0.00044879 -0.01083876] 	 [0 1] 	 [0.50259751 0.49740252]
[0.00013163 0.00874004] 	 [0 1] 	 [0.49784794 0.50215214]
[ 0.00079006 -0.00861552] 	 [0 1] 	 [0.50235140 0.49764863]


[-0.00035114 -0.06397165] 	 [0 1] 	 [0.51589972 0.48410025]
[ 0.00056738 -0.03663344] 	 [1 0] 	 [0.50929916 0.49070087]
[ 0.00056098 -0.02617791] 	 [1 0] 	 [0.5066843 0.4933157]
[-0.00044851 -0.07137010] 	 [0 1] 	 [0.51772296 0.48227707]
[ 0.00013184 -0.05879313] 	 [0 1] 	 [0.514727 0.485273]
[ 0.00079021 -0.05743587] 	 [0 1] 	 [0.51455247 0.48544762]


[-0.00035016 -0.06321616] 	 [0 1] 	 [0.51571137 0.48428872]
[ 0.00056696 -0.03072025] 	 [1 0] 	 [0.50782120 0.49217883]
[ 0.00056129 -0.01022454] 	 [1 0] 	 [0.50269639 0.49730355]
[-0.00044983 -0.00021709] 	 [0 1] 	 [0.4999418 0.5000582]
[0.00013071 0.00484093] 	 [1 0] 	 [0.49882248 

#### `get_sample_policy( hard_sampling = True)` : Calls test_sample_policy of network using ARGMAX of logits

In [131]:
policy_softmaxs = environ.get_policy_prob()
hard_policies, logits = environ.get_sample_policy(hard_sampling = True)

for p,l,s in zip(hard_policies, logits, policy_softmaxs) :
    for  p_row, l_row, s_row in zip(p, l, s):
        print( l_row,'\t', p_row, '\t', s_row)
    print('\n')

[-0.0003512  -0.06617914] 	 [1 0] 	 [0.51645106 0.48354897]
[ 0.00056736 -0.04341661] 	 [1 0] 	 [0.51099426 0.48900577]
[ 0.00056091 -0.01096974] 	 [1 0] 	 [0.50288266 0.49711737]
[-0.00044879 -0.01083876] 	 [1 0] 	 [0.5025975  0.49740252]
[0.00013163 0.00874004] 	 [0 1] 	 [0.49784794 0.50215214]
[ 0.00079006 -0.00861552] 	 [1 0] 	 [0.5023514  0.49764863]


[-0.00035114 -0.06397165] 	 [1 0] 	 [0.5158997  0.48410025]
[ 0.00056738 -0.03663344] 	 [1 0] 	 [0.50929916 0.49070087]
[ 0.00056098 -0.02617791] 	 [1 0] 	 [0.5066843 0.4933157]
[-0.00044851 -0.0713701 ] 	 [1 0] 	 [0.51772296 0.48227707]
[ 0.00013184 -0.05879313] 	 [1 0] 	 [0.514727 0.485273]
[ 0.00079021 -0.05743587] 	 [1 0] 	 [0.5145525  0.48544762]


[-0.00035016 -0.06321616] 	 [1 0] 	 [0.51571137 0.48428872]
[ 0.00056696 -0.03072025] 	 [1 0] 	 [0.5078212  0.49217883]
[ 0.00056129 -0.01022454] 	 [1 0] 	 [0.5026964  0.49730355]
[-0.00044983 -0.00021709] 	 [0 1] 	 [0.4999418 0.5000582]
[0.00013071 0.00484093] 	 [0 1] 	 [0.49882248 

#### Print

In [135]:
print(f" Layer    task 1      task 2      task 3")
print(f" -----    ------      ------      ------")
for idx, (l1, l2, l3) in enumerate(zip(hard_policies[0], hard_policies[1], hard_policies[2]),1):
    print(f"   {idx}      {l1}       {l2}       {l3}")
    

    print(f"\n\n where [p1  p2]:  p1: layer is selected    p2: layer is not selected")

 Layer    task 1      task 2      task 3
 -----    ------      ------      ------
   1      [1 0]       [1 0]       [1 0]
   2      [1 0]       [1 0]       [1 0]
   3      [1 0]       [1 0]       [1 0]
   4      [1 0]       [1 0]       [0 1]
   5      [0 1]       [1 0]       [0 1]
   6      [1 0]       [1 0]       [1 0]


 where [p1  p2]:  p1: layer is selected    p2: layer is not selected


In [402]:
def display_trained_policy(iter):

    policy_softmaxs = environ.get_policy_prob()
    policy_argmaxs = 1-np.argmax(policy_softmaxs, axis = -1)
    print(f"  Trained polcies at iteration: {iter} ")
    print(f"                   task 1                           task 2                         task 3        ")
    print(f" Layer       softmax        select          softmax        select          softmax        select   ")
    print(f" -----    ---------------   ------       ---------------   ------       ---------------   ------   ")
    for idx, (l1,l2,l3,  p1,p2,p3) in enumerate(zip(policy_softmaxs[0], policy_softmaxs[1], policy_softmaxs[2], policy_argmaxs[0], policy_argmaxs[1], policy_argmaxs[2]),1):
        print(f"   {idx}      {l1[0]:.4f}   {l1[1]:.4f}   {p1:4d}    {l2[0]:11.4f}   {l2[1]:.4f}   {p2:4d}    {l3[0]:11.4f}   {l3[1]:.4f}   {p3:4d}")

    print()
# print(f"\n\n where [p1  p2]:  p1: layer is selected    p2: layer is not selected")

In [405]:
display_trained_policy(5)

  Trained polcies at iteration: 5 
                   task 1                           task 2                         task 3        
 Layer       softmax        select          softmax        select          softmax        select   
 -----    ---------------   ------       ---------------   ------       ---------------   ------   
   1      0.5165   0.4835      1         0.5159   0.4841      1         0.5157   0.4843      1
   2      0.5110   0.4890      1         0.5093   0.4907      1         0.5078   0.4922      1
   3      0.5029   0.4971      1         0.5067   0.4933      1         0.5027   0.4973      1
   4      0.5026   0.4974      1         0.5177   0.4823      1         0.4999   0.5001      0
   5      0.4978   0.5022      0         0.5147   0.4853      1         0.4988   0.5012      0
   6      0.5024   0.4976      1         0.5146   0.4854      1         0.5058   0.4942      1



In [355]:
print(f"                        POLICIES (SOFTMAX)                                       task 3          ")
print(f" Layer    task1              task2            task3 softmax         softmax         argmax         softmax         argmax   ")
print(f" -----    -------------     -------------     -------------   ------   ")
for idx, (l1,l2,l3, h1,h2,h3) in enumerate(zip(policy_softmaxs[0], policy_softmaxs[1], policy_softmaxs[2],hard_policies[0], hard_policies[1], hard_policies[2]),1):
    print(f"   {idx}      {l1[0]:.4f} {l1[1]:.4f}     {l2[0]:.4f} {l2[1]:.4f}     {l3[0]:.4f} {l3[1]:.4f}    {h3}")
    
print(f"\n\n where [p1  p2]:  p1: layer is selected    p2: layer is not selected")

                        POLICIES (SOFTMAX)                                       task 3          
 Layer    task1              task2            task3 softmax         softmax         argmax         softmax         argmax   
 -----    -------------     -------------     -------------   ------   
   1      0.5165 0.4835     0.5159 0.4841     0.5157 0.4843    [1 0]
   2      0.5110 0.4890     0.5093 0.4907     0.5078 0.4922    [1 0]
   3      0.5029 0.4971     0.5067 0.4933     0.5027 0.4973    [1 0]
   4      0.5026 0.4974     0.5177 0.4823     0.4999 0.5001    [0 1]
   5      0.4978 0.5022     0.5147 0.4853     0.4988 0.5012    [0 1]
   6      0.5024 0.4976     0.5146 0.4854     0.5058 0.4942    [1 0]


 where [p1  p2]:  p1: layer is selected    p2: layer is not selected


In [349]:
# print(policy_softmaxs[2], np.argmax(1-policy_softmaxs[2], axis = -1))
print(policy_softmaxs, np.argmax(policy_softmaxs, axis = -1))

[array([[0.51645106, 0.48354897],
       [0.51099426, 0.48900577],
       [0.50288266, 0.49711737],
       [0.50259751, 0.49740252],
       [0.49784794, 0.50215214],
       [0.50235140, 0.49764863]], dtype=float32), array([[0.51589972, 0.48410025],
       [0.50929916, 0.49070087],
       [0.50668430, 0.49331570],
       [0.51772296, 0.48227707],
       [0.51472700, 0.48527300],
       [0.51455247, 0.48544762]], dtype=float32), array([[0.51571137, 0.48428872],
       [0.50782120, 0.49217883],
       [0.50269639, 0.49730355],
       [0.49994180, 0.50005817],
       [0.49882248, 0.50117755],
       [0.50577444, 0.49422547]], dtype=float32)] [[0 0 0 0 1 0]
 [0 0 0 0 0 0]
 [0 0 0 1 1 0]]


#### `get_current_logits()` : Calls test_sample_policy of network using ARGMAX of logits

In [149]:
logits  = (environ.get_current_logits())
for i in logits:
    print(i ,'\n')

[[-0.0003512  -0.06617914]
 [ 0.00056736 -0.04341661]
 [ 0.00056091 -0.01096974]
 [-0.00044879 -0.01083876]
 [ 0.00013163  0.00874004]
 [ 0.00079006 -0.00861552]] 

[[-0.00035114 -0.06397165]
 [ 0.00056738 -0.03663344]
 [ 0.00056098 -0.02617791]
 [-0.00044851 -0.0713701 ]
 [ 0.00013184 -0.05879313]
 [ 0.00079021 -0.05743587]] 

[[-0.00035016 -0.06321616]
 [ 0.00056696 -0.03072025]
 [ 0.00056129 -0.01022454]
 [-0.00044983 -0.00021709]
 [ 0.00013071  0.00484093]
 [ 0.00078938 -0.02230957]] 



#### `get_current_policy()` : Calls test_sample_policy of network using ARGMAX of logits

In [107]:
pols  = (environ.get_current_policy())

for i in pols:
    print(i ,'\n')

[[1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]] 

[[0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]] 

[[0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]] 



#### `gumbel_softmax()`  

In [170]:
np.set_printoptions(precision=8,edgeitems=3, infstr='inf', linewidth=150, nanstr='nan', floatmode = 'maxprec_equal')
torch.set_printoptions(precision=8,linewidth=132)

In [319]:
print(environ.temp)
# tau = environ.temp
tau = 1
for i in range(3): 
    logits_tensor = torch.tensor(logits[0])
    # Sample soft categorical using reparametrization trick:
    gumbel_soft = F.gumbel_softmax(logits_tensor, tau=tau, hard=False).cpu().numpy() 

    # Sample hard categorical using "Straight-through" trick:
    gumbel_hard  = F.gumbel_softmax(logits_tensor, tau=tau, hard=True).cpu().numpy()
    
    for l, gs, gh in zip(lgts, gumbel_soft, gumbel_hard):
        print(f"   {l}   \t {gs}            \t {gh}")
#     print(lgts)
#     print(gumbel_soft)
#     print(gumbel_hard)
    print()

0.0001180506617226113
   [-0.00035016 -0.06321616]   	 [0.17155227 0.82844770]            	 [1. 0.]
   [ 0.00056696 -0.03072025]   	 [0.04782803 0.95217192]            	 [1. 0.]
   [ 0.00056129 -0.01022454]   	 [0.52678031 0.47321963]            	 [0. 1.]
   [-0.00044983 -0.00021709]   	 [0.74226642 0.25773358]            	 [0. 1.]
   [0.00013071 0.00484093]   	 [0.81233245 0.18766758]            	 [1. 0.]
   [ 0.00078938 -0.02230957]   	 [0.76270294 0.23729712]            	 [1. 0.]

   [-0.00035016 -0.06321616]   	 [0.01975500 0.98024493]            	 [0. 1.]
   [ 0.00056696 -0.03072025]   	 [0.33801472 0.66198522]            	 [1. 0.]
   [ 0.00056129 -0.01022454]   	 [0.2644645 0.7355355]            	 [0. 1.]
   [-0.00044983 -0.00021709]   	 [0.08984101 0.91015899]            	 [1. 0.]
   [0.00013071 0.00484093]   	 [0.17066659 0.82933342]            	 [1. 0.]
   [ 0.00078938 -0.02230957]   	 [0.74648136 0.25351864]            	 [0. 1.]

   [-0.00035016 -0.06321616]   	 [0.5077298 0.

In [151]:
for lgts in logits:
    logits_tensor = torch.tensor(lgts)
    print(lgts)
    # Sample soft categorical using reparametrization trick:
    gumbel_soft = F.gumbel_softmax(logits_tensor, tau=1, hard=False)
    print(gumbel_soft)

    # Sample hard categorical using "Straight-through" trick:
    gumbel_hard  = F.gumbel_softmax(logits_tensor, tau=1, hard=True)
    print(gumbel_hard)
    print()

[[-0.0003512  -0.06617914]
 [ 0.00056736 -0.04341661]
 [ 0.00056091 -0.01096974]
 [-0.00044879 -0.01083876]
 [ 0.00013163  0.00874004]
 [ 0.00079006 -0.00861552]]
tensor([[0.22969657, 0.77030343],
        [0.47433791, 0.52566212],
        [0.60556847, 0.39443150],
        [0.00808809, 0.99191189],
        [0.99667323, 0.00332679],
        [0.56034184, 0.43965816]])
tensor([[0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.]])

[[-0.00035114 -0.06397165]
 [ 0.00056738 -0.03663344]
 [ 0.00056098 -0.02617791]
 [-0.00044851 -0.0713701 ]
 [ 0.00013184 -0.05879313]
 [ 0.00079021 -0.05743587]]
tensor([[0.29727638, 0.70272362],
        [0.65075004, 0.34924990],
        [0.83831531, 0.16168469],
        [0.72130281, 0.27869719],
        [0.87410325, 0.12589674],
        [0.53555954, 0.46444049]])
tensor([[0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.]])

[[-0.00035016 -0.06321616]
 [ 0.00056696 -0.

In [33]:
smax = scipy.special.softmax(logs, axis =1)
# smax = np.array( 
# [[0.46973792, 0.530262  ],
#  [0.45025694, 0.549743  ],
#  [0.4443086 , 0.5556915 ],
#  [0.4138397 , 0.58616036],
#  [0.4140113 , 0.5859887 ],
#  [0.42114905, 0.57885087]])

print(smax.shape)
print(smax)
print(smax[0])
print(smax[0].sum())
print(np.random.choice((1,0), p =smax[0]))

(6, 2)
[[0.47754285 0.52245715]
 [0.45825934 0.54174066]
 [0.45530966 0.54469034]
 [0.43196854 0.56803146]
 [0.43017322 0.56982678]
 [0.43333559 0.56666441]]
[0.47754285 0.52245715]
0.9999999999999998
0


In [142]:
logs = np.array(
[[0.33064184, 0.42053092],
 [0.3532089 , 0.52056104],
 [0.3888512 , 0.5680909 ],
 [0.42039296, 0.694217  ],
 [0.4519742 , 0.73311865],
 [0.48401102, 0.7522658 ]],
)

## Load previously saved model

In [6]:
input_args = " --config yamls/adashare/chembl_2task.yml --cpu --batch_size 09999".split()
# get command line arguments
args = get_command_line_args(input_args)
print(args)

print()

if args.exp_instance is None:
    args.exp_instance = datetime.now().strftime("%m%d_%H%M")
    
print(args.exp_instance, args.config)

 command line parms :  {'config': 'yamls/adashare/chembl_2task.yml', 'exp_instance': None, 'exp_ids': [0], 'batch_size': 9999, 'backbone_lr': None, 'task_lr': None, 'decay_lr_rate': None, 'decay_lr_freq': None, 'gpus': [0], 'cpu': True}
Namespace(config='yamls/adashare/chembl_2task.yml', exp_instance=None, exp_ids=[0], batch_size=9999, backbone_lr=None, task_lr=None, decay_lr_rate=None, decay_lr_freq=None, gpus=[0], cpu=True)

0119_1230 yamls/adashare/chembl_2task.yml


In [7]:
print_separator('READ YAML')
opt, gpu_ids, exp_ids =read_yaml_from_input(args)
print(gpu_ids, exp_ids,  opt['train']['policy_iter'])

##################################################
####################READ YAML#####################
##################################################
[0] [0] best


In [ ]:
current_iter = environ.load_checkpoint('latest')

print('Evaluating the snapshot saved at %d iter' % current_iter)

In [ ]:
opt['train']['weight_iter_alternate'] = opt['train'].get('weight_iter_alternate', len(train1_loader))
opt['train']['alpha_iter_alternate'] = opt['train'].get('alpha_iter_alternate'  , len(train2_loader))

print(opt['train']['weight_iter_alternate'], opt['train']['alpha_iter_alternate'])

## Softmax & Gumbel Softmax

###  Softmax, LogSoftMax, NegLogLikelihood and Cross Entropy

In [4]:
from torch import nn
# print(nn.CrossEntropyLoss.__doc__)
loss = nn.CrossEntropyLoss(reduction ='none')
sm = nn.Softmax(dim =-1)
lsm = nn.LogSoftmax(dim = -1)
nll = nn.NLLLoss(reduction='none')

# i1 = torch.randn(3, 5, requires_grad=True)
# t1 = torch.empty(3, dtype=torch.long).random_(5)

# print(i1)
# print(i1)
# print(t1)
# output = loss(i1, t1)
# print(output, output.sum(), output.mean())

# i2 = torch.randn(1, 2, requires_grad=True)
i0 = torch.tensor([[0.0, 1.0]], dtype=torch.float)
i1 = torch.tensor([[1.0, 0.0]], dtype=torch.float)
i2 = torch.tensor([[0.5, 0.5]], dtype=torch.float)
i3 = torch.tensor([[0.4656, 0.5388]], dtype=torch.float)


t1 = torch.tensor([1], dtype=torch.int64)
t0 = torch.tensor([0], dtype=torch.int64)
t2 = torch.tensor([2], dtype=torch.int64)
print('i0     : ', i0)
print('sm(i0) : ', sm(i0))
print('lsm(i0): ', lsm(i0))
print()
print('i1     : ', i1)
print('sm(i1) : ', sm(i1))
print('lsm(i1): ', lsm(i1))
print()
print('i2     : ', i2)
print('sm(i2) : ', sm(i2))
print('lsm(i2): ', lsm(i2))
print()

print('t0: ',t0)
print('t1: ',t1)
print()
output1 = loss(i0, t0)
output2 = nll(lsm(i0), t0)
print('loss [0,1] and [0] : ', output1)
print('nll between lsm(i0): ', output2)
print()
output1 = loss(i0, t1)
output2 = nll(lsm(i0), t1)
print('loss [0,1] and [1] : ', output1)
print('nll between lsm(i0): ', output2)
print()

output1 = loss(i1, t0)
output2 = nll(lsm(i1), t0)
print('loss [1,0] and [0] : ', output1)
print('nll between lsm(i1): ', output2)
print()

output1 = loss(i1, t1)
output2 = nll(lsm(i1), t1)
print('loss [1,0] and [1] : ', output1)
print('nll between lsm(i1): ', output2)
print()

output1 = loss(i2, t0)
output2 = nll(lsm(i2), t0)
print('loss [0.5, 0.5] and [0] : ', output1)
print('nll between lsm(i1)   and [0] : ', output2)
print()

output1 = loss(i2, t1)
output2 = nll(lsm(i2), t1)
print('loss [0.5, 0.5] and [1] : ', output1)
print('nll between lsm(i1)   and [1] : ', output2)
print()


i0     :  tensor([[0., 1.]])
sm(i0) :  tensor([[0.2689, 0.7311]])
lsm(i0):  tensor([[-1.3133, -0.3133]])

i1     :  tensor([[1., 0.]])
sm(i1) :  tensor([[0.7311, 0.2689]])
lsm(i1):  tensor([[-0.3133, -1.3133]])

i2     :  tensor([[0.5000, 0.5000]])
sm(i2) :  tensor([[0.5000, 0.5000]])
lsm(i2):  tensor([[-0.6931, -0.6931]])

t0:  tensor([0])
t1:  tensor([1])

loss [0,1] and [0] :  tensor([1.3133])
nll between lsm(i0):  tensor([1.3133])

loss [0,1] and [1] :  tensor([0.3133])
nll between lsm(i0):  tensor([0.3133])

loss [1,0] and [0] :  tensor([0.3133])
nll between lsm(i1):  tensor([0.3133])

loss [1,0] and [1] :  tensor([1.3133])
nll between lsm(i1):  tensor([1.3133])

loss [0.5, 0.5] and [0] :  tensor([0.6931])
nll between lsm(i1)   and [0] :  tensor([0.6931])

loss [0.5, 0.5] and [1] :  tensor([0.6931])
nll between lsm(i1)   and [1] :  tensor([0.6931])



### Gumbel Softmax

In [137]:
a = torch.tensor(
       [[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000]], device='cuda:0', requires_grad=True) 

b = torch.tensor([0.5000, 0.5000,  0.5000, 0.5000], device='cuda:0', requires_grad=True) 
print(b.shape)
c = torch.tensor([[0.000, 0.000,  0.000, 0.000]], device='cuda:0', requires_grad=True) 
print(c.shape)
d = torch.tensor([[0.5000], [0.5000],  [0.5000], [0.5000]], device='cuda:0', requires_grad=True) 
print(d.shape)

torch.Size([4])
torch.Size([1, 4])
torch.Size([4, 1])


In [138]:
i0 = torch.tensor([[0.0, 1.0]], dtype=torch.float)
i1 = torch.tensor([[1.0, 0.0]], dtype=torch.float)
i2 = torch.tensor([[0.5, 0.5]], dtype=torch.float)
i3 = torch.tensor([[0.4656, 0.5388]], dtype=torch.float)


print(i0)
print(i1)
print(i2)
print(i3)

tensor([[0., 1.]])
tensor([[1., 0.]])
tensor([[0.50000000, 0.50000000]])
tensor([[0.46560001, 0.53880000]])


In [139]:
temp  = 2.5
print(i0)
print('-'*25)
print(F.gumbel_softmax( i0, temp, hard=False))
print(F.gumbel_softmax( i0, temp, hard=False))
print(F.gumbel_softmax( i0, temp, hard=False))
print()

print(i1)
print('-'*25)
print(F.gumbel_softmax( i1, temp, hard=False))
print(F.gumbel_softmax( i1, temp, hard=False))
print(F.gumbel_softmax( i1, temp, hard=False))
print()

print(i2)
print('-'*25)
print(F.gumbel_softmax( i2, temp, hard=False))
print(F.gumbel_softmax( i2, temp, hard=False))
print(F.gumbel_softmax( i2, temp, hard=False))
print()


print(i3)
print('-'*25)
print(F.gumbel_softmax( i3, temp, hard=False))
print(F.gumbel_softmax( i3, temp, hard=False))
print(F.gumbel_softmax( i3, temp, hard=False))
# print(F.gumbel_softmax( d, 5, hard=False))

tensor([[0., 1.]])
-------------------------
tensor([[0.35299182, 0.64700818]])
tensor([[0.44149891, 0.55850112]])
tensor([[0.20112066, 0.79887938]])

tensor([[1., 0.]])
-------------------------
tensor([[0.66644371, 0.33355635]])
tensor([[0.49296239, 0.50703758]])
tensor([[0.63502610, 0.36497390]])

tensor([[0.50000000, 0.50000000]])
-------------------------
tensor([[0.45357519, 0.54642475]])
tensor([[0.53368020, 0.46631980]])
tensor([[0.53128326, 0.46871671]])

tensor([[0.46560001, 0.53880000]])
-------------------------
tensor([[0.75833476, 0.24166526]])
tensor([[0.47358128, 0.52641875]])
tensor([[0.47027132, 0.52972871]])


In [ ]:
logits = torch.randn(20, 32)

In [ ]:
print(logits[:2,:])

In [ ]:
tmp = -torch.empty_like(logits, memory_format=torch.legacy_contiguous_format)
print(tmp[0])

In [ ]:
print(tmp[0])
tmp1 = tmp.exponential_()
print(tmp1[0])

In [ ]:
tmp2 = tmp1.log()
print(tmp2[0])

In [ ]:
 gumbels = (
        -torch.empty_like(logits, memory_format=torch.legacy_contiguous_format).exponential_().log()
    ) 

In [ ]:
print(gumbels.shape)
print(gumbels[0])

In [140]:
# Sample soft categorical using reparametrization trick:
gumbel_soft = F.gumbel_softmax(logits, tau=1, hard=False)

# Sample hard categorical using "Straight-through" trick:
gumbel_hard  = F.gumbel_softmax(logits, tau=1, hard=True)

TypeError: empty_like(): argument 'input' (position 1) must be Tensor, not list

In [ ]:
print(logits.shape)
print(logits[0])
print(np.argmax(logits[0]))
print('\n')

print(gumbel_soft.shape)
print(gumbel_soft[0])
print(np.argmax(gumbel_soft[0]))
print('\n')

print(gumbel_hard.shape)
print(gumbel_hard[0])
print(np.argmax(gumbel_hard[0]))

In [ ]:
gumbel_soft.sum(axis=1)

In [ ]:
tau = 1

In [ ]:
a = torch.empty_like(logits, memory_format=torch.legacy_contiguous_format)

In [ ]:
a[:2]

fill tensor `a` with elements drawn from exponential distribution

In [ ]:
a_e = a.exponential_()

In [ ]:
a_e[:2]

draw natural log `ln()` on elements of a_e

In [ ]:
a_e_l = a_e.log()

In [ ]:
a_e_l[:2]

Neg log

In [ ]:
a_el_neg = -a_e_l

a_el_neg[:2]

In [ ]:
logits[:2]

In [ ]:
gumbels = (logits + a_el_neg) / tau 

gumbels[:2]

In [ ]:
dim = -1
gumbels.shape

In [ ]:
 y_soft = gumbels.softmax(dim)

In [ ]:
y_soft.shape

In [ ]:
y_soft[:2]

In [ ]:
index = y_soft.max(dim, keepdim=True)
print(index[0].T)
print(index[1].T)
y_hard = torch.zeros_like(logits, memory_format=torch.legacy_contiguous_format).scatter_(dim, index[1], 1.0)

In [ ]:
np.argmax(y_hard,axis=1)

In [ ]:
tmp_d= [0,1,0]
for i in range(10):
    sampled = np.random.choice((2, 1, 0), p=tmp_d)
    print(sampled)

## Scratch Pad

In [3]:
# from numba import cuda

# cuda_device = 0 

# def free_gpu_cache(cuda_device):
#     print("Initial GPU Usage")    
#     gpu_usage()                             
#     print("GPU Usage after emptying the cache")
#     gpu_usage()
#     print("CUDA empty cache")
#     torch.cuda.empty_cache()
#     print("Close and reopen device")
#     cuda.select_device(cuda_device)
#     print("Close device")    
#     cuda.close()
#     print("Reopen device")    
#     cuda.select_device(cuda_device)
#     print("GPU Usage after closing and reopening")
#     gpu_usage()

# free_gpu_cache(0)

In [7]:
# def print_separator(text, total_len=50):
#     print('#' * total_len)
#     left_width = (total_len - len(text))//2
#     right_width = total_len - len(text) - left_width
#     print("#" * left_width + text + "#" * right_width)
#     print('#' * total_len)

# def print_dbg(text, verbose = False):
#     if verbose:
#         print(text)

# @debug_off
# def print_heading(text,  verbose = False):
#     len_ttl = max(len(text)+4, 50)
#     if verbose:
#         print('-' * len_ttl)
#         print(f" {text}")
#         # left_width = (total_len - len(text))//2
#         # right_width = total_len - len(text) - left_width
#         # print("#" * left_width + text + "#" * right_width)
#         print('-' * len_ttl,'\n')

# print_heading("hello_kevin", verbose=False)

### Chembl Data feed

In [5]:
# dataroot = opt['dataload']['dataroot']
# ecfp     = load_sparse(dataroot, opt['dataload']['x'])

# total_input = ecfp.shape[0]
# ranges      = (np.cumsum([0]+opt['dataload']['x_split_ratios'])* total_input).astype(np.int32)


# idx_train  = np.arange(ranges[0], ranges[1])
# idx_train1 = np.arange(ranges[1], ranges[2])
# idx_train2 = np.arange(ranges[2], ranges[3])
# idx_val    = np.arange(ranges[3], ranges[4])

# print(f" Total input    :  {total_input}   Cummulative dataset sizes: {ranges}")
# print(f" Ranges         :  {ranges}")
# print()
# print(f" X Dataset      :  {os.path.join(opt['dataload']['dataroot'], opt['dataload']['x'])}")
# print(f" y Dataset      :  {os.path.join(opt['dataload']['dataroot'], opt['dataload']['y_tasks'][0])}")
# print(f" Folding Dataset:  {os.path.join(opt['dataload']['dataroot'], opt['dataload']['folding'])}")
# print(f" Weights_class  :  {opt['dataload']['weights_class']}")
# print()
# print(f' idx_train    dataset size: {len(idx_train)  :6d}  - rows: {(idx_train)} ')
# print(f' idx_train1   dataset size: {len(idx_train1) :6d}  - rows: {(idx_train1)} ')
# print(f' idx_train2   dataset size: {len(idx_train2) :6d}  - rows: {(idx_train2)} ')
# print(f' val_train    dataset size: {len(idx_val)    :6d}  - rows: {(idx_val)} ')

### tqdm

In [ ]:
from tqdm.notebook import tnrange

In [ ]:
curr_iter_t  = 0
curr_iter_a  = 0
curr_iter_w  = 0
stop_iter_t  = 0
stop_iter_w  = 0 
stop_iter_a  = 0
total_weight_epochs = 0
total_policy_epochs = 0 
train_total_iters = 8
weight_iter_alternate = 17
alpha_iter_alternate = 17

In [ ]:
print(curr_iter_t, stop_iter_t, flag, train_total_iters,opt['train']['print_freq'] )
start_iter_t = curr_iter_t
stop_iter_t = curr_iter_t +  train_total_iters 
print(f" Current iteration {curr_iter_t} - Run  from {start_iter_t} to {stop_iter_t}")

print(curr_iter_w, weight_iter_alternate , flag)
stop_iter_w = curr_iter_w +  weight_iter_alternate 
print(f" Current Weight iteration {curr_iter_w} - Run  from {curr_iter_w+1} to {stop_iter_w}")


print(curr_iter_a ,  alpha_iter_alternate ,flag)
stop_iter_a = curr_iter_a +  alpha_iter_alternate 
print(f" Current alpha iteration {curr_iter_a} - Run  from {curr_iter_a+1} to {stop_iter_a}")

In [ ]:
# del t, t_w, t_a
main_iter_ctr = 0 
with tnrange(start_iter_t , stop_iter_t  , initial = start_iter_t , total = stop_iter_t, position=0, leave= True, desc="master") as t :
    for curr_t in t:
        
        with  tnrange(0, weight_iter_alternate , initial = 0, total = weight_iter_alternate, 
                      position=1, leave= False, desc=f"epoch {curr_t} weight training") as t_w :
            for curr_w in t_w:    
                sleep(0.35)
                main_iter_ctr += 1
                curr_iter_w  = curr_w
                t.set_postfix({'epoch': f"{curr_t}/{train_total_iters}", 'main_iter_ctr': main_iter_ctr})
                t_w.set_postfix({'weight training epoch': curr_t, 'batch #': curr_iter_w})

            print(f"** Epoch {curr_t}/{train_total_iters} weight training complete - Loss: "
                  f"curr_w:{curr_w}    curr_iter_w:{curr_iter_w}  curr_t:{curr_t}  main_iter_ctr:{main_iter_ctr}" )
                 
        
        with  tnrange(0, alpha_iter_alternate  , initial = 0, total = alpha_iter_alternate , 
                      position=2, leave= False, desc=f"epoch {curr_t} policy training") as t_a :
            for curr_a in t_a:    
                sleep(0.35)
                main_iter_ctr += 1                
                curr_iter_a = curr_a
                t.set_postfix({'epoch': f"{curr_t}/{train_total_iters}", 'main_iter_ctr':main_iter_ctr})
                t_a.set_postfix({'policy training epoch': curr_t, 'batch #': curr_iter_a})            
                
            print(f"** Epoch {curr_t}/{train_total_iters} policy training complete - Loss: "
                  f"curr_w:{curr_w}    curr_iter_w:{curr_iter_w}  curr_t:{curr_t}  main_iter_ctr:{main_iter_ctr}" )
        

In [ ]:
curr_iter_t  = 0
curr_iter_a  = 0
curr_iter_w  = 0
stop_iter_t  = 0
stop_iter_w  = 0 
stop_iter_a  = 0
total_weight_epochs = 0
total_policy_epochs = 0 
train_total_iters = 100
train_total_epochs = 10
weight_iter_alternate = 17
alpha_iter_alternate = 17

print(curr_iter_t, stop_iter_t, flag, train_total_iters,opt['train']['print_freq'] )
start_iter_t = curr_iter_t
stop_iter_t = curr_iter_t +  train_total_iters 
print(f" Current iteration {curr_iter_t} - Run  from {start_iter_t} to {stop_iter_t}")

print(curr_iter_w, weight_iter_alternate , flag)
stop_iter_w = curr_iter_w +  weight_iter_alternate 
print(f" Current Weight iteration {curr_iter_w} - Run  from {curr_iter_w+1} to {stop_iter_w}")


print(curr_iter_a ,  alpha_iter_alternate ,flag)
stop_iter_a = curr_iter_a +  alpha_iter_alternate 
print(f" Current alpha iteration {curr_iter_a} - Run  from {curr_iter_a+1} to {stop_iter_a}")

In [ ]:
# del t, t_w, t_a
current_epoch = 0
main_iter_ctr = 0 
# with tnrange(start_iter_t , stop_iter_t  , initial = start_iter_t , total = stop_iter_t, position=0, leave= True, desc="master") as t :
# with tqdm_notebook(total=train_total_epochs) as t:
t = tqdm_notebook(total=train_total_epochs)

while current_epoch < train_total_epochs:
    current_epoch+=1
    t.update(1)

    #-----------------------------------------
    # Train & Update the network weights
    #-----------------------------------------        
    with  tnrange(0, weight_iter_alternate , initial = 0, total = weight_iter_alternate, 
                  position=1, leave= False, desc=f"epoch {current_epoch} weight training") as t_w :
        for curr_w in t_w:    
            sleep(0.35)
            main_iter_ctr += 1
            curr_iter_w  = curr_w

            t.set_postfix({'epoch': f"{current_epoch}/{train_total_epochs}", 'main_iter_ctr': main_iter_ctr})
            t_w.set_postfix({'weight training epoch': current_epoch, 'batch #': curr_iter_w})

        tqdm.write(f"** Epoch {current_epoch}/{train_total_epochs} weight training complete - Loss: "
              f"curr_w:{curr_w}    curr_iter_w:{curr_iter_w}  current_epoch:{current_epoch}  main_iter_ctr:{main_iter_ctr}" )

    #-----------------------------------------
    # Train & Update the  policy 
    #-----------------------------------------        
    with  tnrange(0, alpha_iter_alternate  , initial = 0, total = alpha_iter_alternate , 
                  position=2, leave= False, desc=f"epoch {current_epoch} policy training") as t_a :
        for curr_a in t_a:    
            sleep(0.35)
            main_iter_ctr += 1                
            curr_iter_a = curr_a

            t.set_postfix({'epoch': f"{current_epoch}/{train_total_epochs}", 'main_iter_ctr':main_iter_ctr})
            t_a.set_postfix({'policy training epoch': current_epoch, 'batch #': curr_iter_a})            

        tqdm.write(f"** Epoch {current_epoch}/{train_total_epochs} policy training complete - Loss: "
              f"curr_w:{curr_w}    curr_iter_w:{curr_iter_w}  current_epoch:{current_epoch}  main_iter_ctr:{main_iter_ctr}" )


In [ ]:
# with tnrange(start_iter, stop_iter , initial = current_iter_w, total = stop_iter,  position=0, leave= True, desc="training") as t:
#     for current_iter_w in t:
#         print(current_iter_w)
#         current_iter_w += 1
#         print(current_iter_w)        

In [ ]:
# start = current_iter
# end = current_iter + opt['train']['warm_up_iters']
# curr_range = range(start,end)
# print(start, end)

# for i in tqdm.notebook.tnrange(start, end, initial = start, total = end):
#     sleep(0.25)
#     current_iter += 1
# #     print(i)
#     pass

# print(current_iter)

In [ ]:
# start = current_iter
# end = current_iter + opt['train']['warm_up_iters']
# curr_range = range(start,end)
# print(start, end)

# for i in tqdm.notebook.tqdm_notebook(cur_range, initial = start, total = end, disable=False, position=0, desc = "validation"):
#     current_iter += 1
#     pass

# print(current_iter)

In [ ]:
# from tqdm import trange
# from time import sleep

# for i in trange(40, desc='1st loop', position=0, leave = False):
#     sleep(1.1)
#     for j in trange(5, desc='2nd loop', position =1, leave = False):
#         sleep(0.01)
#         for k in trange(50, desc='3rd loop', position =0,leave=False):
#             sleep(0.01)

In [ ]:
# with tqdm(batch_enumerator, leave=False, disable=False) as t:
# with tqdm(total=10, bar_format="{postfix[0]} {postfix[1][value]:>8.2g}", postfix=["Batch", dict(value=0)]) as t:
# with trange(opt['train']['warm_up_iters'], bar_format="{postfix[0]} {postfix[1][value]:>8.2g}", postfix=["Batch", dict(value=0)]) as t:
# with trange(opt['train']['warm_up_iters']) as t:

#     for current_iter in t:
#         batch_idx, batch = next(batch_enumerator)
#         ran = random.randint(1, 100)
#         start_time = time.time()

#         environ.train()

#         print_heading(f" {timestring()} - WARMUP Training iter {current_iter}/{opt['train']['warm_up_iters']}    batch_idx: {batch_idx}"    
#                       f"    Warm-up iters: {opt['train']['warm_up_iters']}"
#                       f"    Validation freq:  {opt['train']['val_freq']}", verbose = False)

#         if batch_idx == len(train_loader) :
#     #         print_heading(f" ******* {timestring()}  re-enumerate train_loader() *******")
#             batch_enumerator = enumerate(train_loader,1)   

#         t.set_postfix({'batch_idx': batch_idx, 'num_vowels': ran})


In [ ]:
# import tqdm.notebook

### using eval()

In [ ]:
task1 = [1,2,3]
task2 = None
task3 = {'4': 'Kevin', '5':'Bardool'}

for i in [1,2,3]:
    print('task{:d}'.format(i))
    if eval('task{:d}'.format(i)) is None:
        print('task{:d} :  has not been defined '.format(i))
#         exec_str = 'task{:d} =  np.random.rand({:d},{:d})'.format(i,3,2)
        exec_str = 'y_task{:d} = scipy.sparse.csr_matrix(({:d}, {:d})) '.format(i,3,2)
        print(exec_str)
        exec(exec_str)
        print('task{:d} : '.format(i), eval('task{:d}'.format(i)))
        print(eval('type(task{:d})'.format(i)))
    else:
        print('task{:d} : '.format(i), eval('task{:d}'.format(i)))
        
print(f"Created task{i} shape        : {eval('len(task{:d})'.format(i))}")
print(len(task3))

In [ ]:
a = np.random.rand(3,2)
print(a)

In [ ]:
print(eval('len(task{:d})'.format(i)))

## Dev code

### Warm-up:  validation - Dev

In [ ]:
# validation
if should(current_iter, opt['train']['val_freq']):
    print(f"**  {timestring()}  START VALIDATION iteration: {current_iter} ")    

    environ.eval()     # set to evaluation mode (train = False)
    num_seg_class = opt['tasks_num_class'][opt['tasks'].index('seg')] if 'seg' in opt['tasks'] else -1
    val_metrics = eval_dev(environ, 
                          val_loader, 
                          opt['tasks'], 
                          policy=False, 
                          num_train_layers=None, 
                          eval_iter = 4)

In [ ]:
val_metrics.keys()
val_metrics['loss']

In [ ]:
for i in val_metrics:
    print(f'\n {i} \n -----------------')
    print(val_metrics[i])

In [ ]:
    for t_id, task in enumerate(environ.tasks):
        task_key = f"task{t_id+1}"    
        environ.print_loss(current_iter, start_time, val_metrics[task_key]["classification_agg"], title='validation')
    

In [ ]:
    environ.save_checkpoint('latest', current_iter)

    print(f"** {timestring()} - END VALIDATION iteration:  {current_iter} ")                
    environ.train()    # set to training mode (train = True)

In [ ]:
# for i in val_metrics.keys():
#     print(i, type(val_metrics[i]))
#     for k in val_metrics[i].keys():
#         print(i,k, type(val_metrics[i][k]))
#         if isinstance(val_metrics[i][k], pd.core.series.Series):
#             print(f"val_metrics[{i}][{k}] is a series")
#         elif isinstance(val_metrics[i][k], pd.core.frame.DataFrame):
#             print(f"val_metrics[{i}][{k}] is a dataframe")        

# s = val_metrics['task1']['classification_agg']
# print(s)
# print(s.to_dict())

In [24]:
##
##  Iteration based method
##
# ##---------------------------------------------------------------     
# ## part one: warm up
# ##---------------------------------------------------------------
# num_prints = 0
# print(f" Last iteration: {current_iter}  # of warm-up iterations to do:{opt['train']['warm_up_iters']} - Run  from {current_iter+1} to {stop_iter}")
# t0 = time.time()

# with trange(current_iter+1, stop_iter+1 , initial = current_iter, total = stop_iter, position=0, leave= True, desc="training") as t_warmup :
    
#     for current_iter in t_warmup:
#         start_time = time.time()
#         environ.train()
#         batch = next(train_loader)    
    
# #         print_heading(f" {timestring()} - WARMUP Training iter {current_iter}/{opt['train']['warm_up_iters']} ", verbose = False)

#         environ.set_inputs(batch, train_loader.dataset.input_size)

#         environ.optimize(opt['lambdas'], 
#                          is_policy=False, 
#                          flag='update_w', 
#                          verbose = False)
        
#         t_warmup.set_postfix({'curr_iter':current_iter, 
#                               'Loss': f"{environ.losses['total']['total'].item():.4f}" , 
#                               'row_ids':f"{batch['row_id'][0]}-{batch['row_id'][-1]}"})
        
# ## Using this causes displaying a zig zag pattern in the training losses 
# #         if should(current_iter, opt['train']['print_freq']):
# #             environ.print_loss(current_iter, start_time, title = f"[c]Warmup training : iteration: {current_iter}", verbose = True)
    
#         ##--------------------------------------------------------------- 
#         # validation
#         ##--------------------------------------------------------------- 
#         if should(current_iter, opt['train']['val_freq']):
#             environ.print_loss(current_iter, start_time, title = f"[e]Weight training epoch: iteration: {current_iter}", verbose = True)

#             val_metrics = environ.evaluate(
#                                    val_loader, 
#                                    opt['tasks'], 
#                                    is_policy=False, 
#                                    num_train_layers=None,
#                                    eval_iters = eval_iters, 
#                                    progress=True,
#                                    leave = False,
#                                    verbose = False)

#             environ.print_metrics(current_iter, start_time, title='validation')
#             environ.save_checkpoint('warmup', current_iter)
            
#             print_metrics_cr(current_iter, time.time() - t0, None, environ.val_metrics, num_prints)
#             num_prints += 1            
#             t0 = time.time()
#             print()     

### Save Best Checkpoint Code

In [ ]:
            #-----------------------------------------------------------------------------------------------------------------------
            #  Save Best Checkpoint Code
            #-----------------------------------------------------------------------------------------------------------------------
            #
            #            #----------------------------------------------------------------------------------------------
            #            # if number of iterations completed after the warm up phase is greater than the number of 
            #            # (weight/policy alternations) x (cirriculum speed) x (number of layers to be policy trained)
            #            #
            #            # check metrics for improvement, and issue a checkpoint if necessary
            #            #----------------------------------------------------------------------------------------------
            # 
            #             if current_iter - opt['train']['warm_up_iters'] >= num_blocks * opt['curriculum_speed'] * \
            #                     (opt['train']['weight_iter_alternate'] + opt['train']['alpha_iter_alternate']):
            #                 new_value = 0
            #                 print(f"  {current_iter - opt['train']['warm_up_iters']} IS GREATER THAN "
            #                        f" {num_blocks * opt['curriculum_speed'] * (opt['train']['weight_iter_alternate'] + opt['train']['alpha_iter_alternate'])} -- "
            #                        f"  evaluate progress and make checkpoint if necessary." )            
            # 
            #                 ## compare validation metrics against reference metrics.
            #                 
            #                 for k in refer_metrics.keys():
            #                     if k in val_metrics.keys():
            #                         for kk in val_metrics[k].keys():
            #                             if not kk in refer_metrics[k].keys():
            #                                 continue
            #                             if (k == 'sn' and kk in ['Angle Mean', 'Angle Median']) or (
            #                                     k == 'depth' and not kk.startswith('sigma')) or (kk == 'err'):
            #                                 value = refer_metrics[k][kk] / val_metrics[k][kk]
            #                             else:
            #                                 value = val_metrics[k][kk] / refer_metrics[k][kk]
            #                             value = value / len(list(set(val_metrics[k].keys()) & set(refer_metrics[k].keys())))
            #                             new_value += value
            # 
            #                 print('Best Value %.4f  New value: %.4f' % new_value)
            # 
            #                 ## if results have improved, save these results and issue a checkpoint
            # 
            #                 if (new_value > best_value):
            #                     print('Previous best iter: %d, best_value: %.4f' % (best_iter, best_value), best_metrics)
            #                     best_value = new_value
            #                     best_metrics = val_metrics
            #                     best_iter = current_iter
            #                     environ.save_checkpoint('best', current_iter)
            #                     print('New      best iter: %d, best_value: %.4f' % (best_iter, best_value), best_metrics)                         
            #                     print('Best Value %.4f  New value: %.4f' % new_value)
            #
            #-----------------------------------------------------------------------------------------------------------------------


### Warm-up:  validation - Dev

In [ ]:
# validation
if should(current_iter, opt['train']['val_freq']):
    print(f"**  {timestring()}  START VALIDATION iteration: {current_iter} ")    

    environ.eval()     # set to evaluation mode (train = False)
    num_seg_class = opt['tasks_num_class'][opt['tasks'].index('seg')] if 'seg' in opt['tasks'] else -1
    val_metrics = eval_dev(environ, 
                          val_loader, 
                          opt['tasks'], 
                          policy=False, 
                          num_train_layers=None, 
                          eval_iter = 4)

In [ ]:
    for t_id, task in enumerate(environ.tasks):
        task_key = f"task{t_id+1}"    
        environ.print_loss(current_iter, start_time, val_metrics[task_key]["classification_agg"], title='validation')
    

In [ ]:
    environ.save_checkpoint('latest', current_iter)

    print(f"** {timestring()} - END VALIDATION iteration:  {current_iter} ")                
    environ.train()    # set to training mode (train = True)

In [ ]:
val_metrics.keys()
val_metrics['loss']

In [ ]:
for i in val_metrics:
    print(f'\n {i} \n -----------------')
    print(val_metrics[i])

In [ ]:
# for i in val_metrics.keys():
#     print(i, type(val_metrics[i]))
#     for k in val_metrics[i].keys():
#         print(i,k, type(val_metrics[i][k]))
#         if isinstance(val_metrics[i][k], pd.core.series.Series):
#             print(f"val_metrics[{i}][{k}] is a series")
#         elif isinstance(val_metrics[i][k], pd.core.frame.DataFrame):
#             print(f"val_metrics[{i}][{k}] is a dataframe")        

# s = val_metrics['task1']['classification_agg']
# print(s)
# print(s.to_dict())

In [24]:
##
##  Iteration based method
##
# ##---------------------------------------------------------------     
# ## part one: warm up
# ##---------------------------------------------------------------
# num_prints = 0
# print(f" Last iteration: {current_iter}  # of warm-up iterations to do:{opt['train']['warm_up_iters']} - Run  from {current_iter+1} to {stop_iter}")
# t0 = time.time()

# with trange(current_iter+1, stop_iter+1 , initial = current_iter, total = stop_iter, position=0, leave= True, desc="training") as t_warmup :
    
#     for current_iter in t_warmup:
#         start_time = time.time()
#         environ.train()
#         batch = next(train_loader)    
    
# #         print_heading(f" {timestring()} - WARMUP Training iter {current_iter}/{opt['train']['warm_up_iters']} ", verbose = False)

#         environ.set_inputs(batch, train_loader.dataset.input_size)

#         environ.optimize(opt['lambdas'], 
#                          is_policy=False, 
#                          flag='update_w', 
#                          verbose = False)
        
#         t_warmup.set_postfix({'curr_iter':current_iter, 
#                               'Loss': f"{environ.losses['total']['total'].item():.4f}" , 
#                               'row_ids':f"{batch['row_id'][0]}-{batch['row_id'][-1]}"})
        
# ## Using this causes displaying a zig zag pattern in the training losses 
# #         if should(current_iter, opt['train']['print_freq']):
# #             environ.print_loss(current_iter, start_time, title = f"[c]Warmup training : iteration: {current_iter}", verbose = True)
    
#         ##--------------------------------------------------------------- 
#         # validation
#         ##--------------------------------------------------------------- 
#         if should(current_iter, opt['train']['val_freq']):
#             environ.print_loss(current_iter, start_time, title = f"[e]Weight training epoch: iteration: {current_iter}", verbose = True)

#             val_metrics = environ.evaluate(
#                                    val_loader, 
#                                    opt['tasks'], 
#                                    is_policy=False, 
#                                    num_train_layers=None,
#                                    eval_iters = eval_iters, 
#                                    progress=True,
#                                    leave = False,
#                                    verbose = False)

#             environ.print_metrics(current_iter, start_time, title='validation')
#             environ.save_checkpoint('warmup', current_iter)
            
#             print_metrics_cr(current_iter, time.time() - t0, None, environ.val_metrics, num_prints)
#             num_prints += 1            
#             t0 = time.time()
#             print()     

### Save Best Checkpoint Code

In [ ]:
            #-----------------------------------------------------------------------------------------------------------------------
            #  Save Best Checkpoint Code
            #-----------------------------------------------------------------------------------------------------------------------
            #
            #            #----------------------------------------------------------------------------------------------
            #            # if number of iterations completed after the warm up phase is greater than the number of 
            #            # (weight/policy alternations) x (cirriculum speed) x (number of layers to be policy trained)
            #            #
            #            # check metrics for improvement, and issue a checkpoint if necessary
            #            #----------------------------------------------------------------------------------------------
            # 
            #             if current_iter - opt['train']['warm_up_iters'] >= num_blocks * opt['curriculum_speed'] * \
            #                     (opt['train']['weight_iter_alternate'] + opt['train']['alpha_iter_alternate']):
            #                 new_value = 0
            #                 print(f"  {current_iter - opt['train']['warm_up_iters']} IS GREATER THAN "
            #                        f" {num_blocks * opt['curriculum_speed'] * (opt['train']['weight_iter_alternate'] + opt['train']['alpha_iter_alternate'])} -- "
            #                        f"  evaluate progress and make checkpoint if necessary." )            
            # 
            #                 ## compare validation metrics against reference metrics.
            #                 
            #                 for k in refer_metrics.keys():
            #                     if k in val_metrics.keys():
            #                         for kk in val_metrics[k].keys():
            #                             if not kk in refer_metrics[k].keys():
            #                                 continue
            #                             if (k == 'sn' and kk in ['Angle Mean', 'Angle Median']) or (
            #                                     k == 'depth' and not kk.startswith('sigma')) or (kk == 'err'):
            #                                 value = refer_metrics[k][kk] / val_metrics[k][kk]
            #                             else:
            #                                 value = val_metrics[k][kk] / refer_metrics[k][kk]
            #                             value = value / len(list(set(val_metrics[k].keys()) & set(refer_metrics[k].keys())))
            #                             new_value += value
            # 
            #                 print('Best Value %.4f  New value: %.4f' % new_value)
            # 
            #                 ## if results have improved, save these results and issue a checkpoint
            # 
            #                 if (new_value > best_value):
            #                     print('Previous best iter: %d, best_value: %.4f' % (best_iter, best_value), best_metrics)
            #                     best_value = new_value
            #                     best_metrics = val_metrics
            #                     best_iter = current_iter
            #                     environ.save_checkpoint('best', current_iter)
            #                     print('New      best iter: %d, best_value: %.4f' % (best_iter, best_value), best_metrics)                         
            #                     print('Best Value %.4f  New value: %.4f' % new_value)
            #
            #-----------------------------------------------------------------------------------------------------------------------


### Policy Training Dev Code

In [ ]:
# print(f" current iter                                 : {current_iter} \n"
#       f" opt['train']['warm_up_iters']                : {opt['train']['warm_up_iters']} \n"
#       f" num_blocks                                   : {num_blocks} \n"
#       f" opt['curriculum_speed']                      : {opt['curriculum_speed']}\n"
#       f" opt['train']['weight_iter_alternate']        : {opt['train']['weight_iter_alternate']}\n"
#       f" opt['train']['alpha_iter_alternate']         : {opt['train']['alpha_iter_alternate']}\n"
#       f" alpha_iter_alternate + weight_iter_alternate : {opt['train']['weight_iter_alternate'] + opt['train']['alpha_iter_alternate']}\n"
#       f" num_blocks * curriculum_speed * (alpha_iter_alternate + weight_iter_alternate): \
#           {num_blocks * opt['curriculum_speed'] * (opt['train']['weight_iter_alternate'] + opt['train']['alpha_iter_alternate'])} \n"
#       f" IF {current_iter - opt['train']['warm_up_iters']} IS GREATER THAN  ??"
#       f" {num_blocks * opt['curriculum_speed'] * (opt['train']['weight_iter_alternate'] + opt['train']['alpha_iter_alternate'])}")


In [ ]:
# print(f" task1_logits: {environ.networks['mtl-net'].task1_logits} \n")
# print(f" task2_logits: {environ.networks['mtl-net'].task2_logits} \n")
# print(f" task3_logits: {environ.networks['mtl-net'].task3_logits} \n")

In [ ]:
# print(current_iter_a , opt['train']['alpha_iter_alternate'],flag)

In [ ]:
##---------------------------------------------------------------     
## part one: warm up
##--------------------------------------------------------------- 
# print(current_iter_a , opt['train']['alpha_iter_alternate'],flag)
# stop_iter = current_iter_a +  opt['train']['alpha_iter_alternate']
# print(f" Run iteration {current_iter_a+1} to {stop_iter+1}")

In [ ]:
# print(current_iter_a, stop_iter, flag)
# print(current_iter_a , opt['train']['alpha_iter_alternate'],flag)

In [ ]:


if flag == 'update_alpha':

    stop_iter = current_iter_a +  opt['train']['alpha_iter_alternate']
    print(f" Current Alpha iteration {current_iter_a} - Run  from {current_iter_a+1} to {stop_iter+1}")
    
    with tnrange(current_iter_a+1, stop_iter+1 , initial = current_iter_a+1, total = stop_iter+1, position=0, leave= True, desc="weight training") as t :
        for current_iter_a in t:    
            current_iter += 1
 
            batch_idx_a, batch = next(batch_enumerator2)
            environ.set_inputs(batch, train2_loader.dataset.input_size)

            if batch_idx_a == len(train2_loader):
                print_dbg(f" Re-enumerate train2_loader  batch_idx_a: {batch_idx_a}   len(train2_loader) = {len(train2_loader)}", verbose=False)                
                batch_enumerator2 = enumerate(train2_loader,1)        
                  
            print_heading(f"{timestring()} - ENVIRON.OPTIMIZE()    flag: {flag}    current_iter: {current_iter}   \n"
                          f" current_iter_w: {current_iter_w}  batch_idx_w:{batch_idx_w}   weight_iter_alternate: {opt['train']['weight_iter_alternate']}\n"
                          f" current_iter_a: {current_iter_a}  batch_idx_a:{batch_idx_a}   alpha_iter_alternate : {opt['train']['alpha_iter_alternate']} \n"
                          f" is_policy: {opt['policy']}   num_train_layers: {num_train_layers}  hard_sampling: {opt['train']['hard_sampling']}\n"
                          f" is_curriculum: {opt['is_curriculum']}     curriculum_speed: {opt['curriculum_speed']}   p_epoch: {p_epoch}"
                          , verbose = False) 
    
            if opt['is_curriculum']:
                num_train_layers = (p_epoch // opt['curriculum_speed']) + 1
            else:
                num_train_layers = None

            print_dbg(f" num_train_layers  : {num_train_layers}", verbose = False)


            environ.optimize(opt['lambdas'], 
                             is_policy=opt['policy'], 
                             flag=flag, 
                             num_train_layers=num_train_layers,
                             hard_sampling=opt['train']['hard_sampling'],
                             verbose = False)

            if should(current_iter, opt['train']['print_freq']):
                environ.print_loss_2(current_iter, start_time, verbose=True)
                environ.resize_results()
                # environ.visual_policy(current_iter)

            print_heading(f"{timestring()} - CONTINUE ALPHA TRAINING    current_iter: {current_iter}\n"
                          f"{' ':15s} current_iter_w: {current_iter_w}  batch_idx_w:{batch_idx_w}   weight_iter_alternate: {opt['train']['weight_iter_alternate']}\n"
                          f"{' ':15s} current_iter_a: {current_iter_a}  batch_idx_a:{batch_idx_a}   alpha_iter_alternate : {opt['train']['alpha_iter_alternate']} ", 
                          verbose = False )      
    
    ## if (current_iter_a % alpha_iter_alternate) == 0 
    if should(current_iter_a, opt['train']['alpha_iter_alternate']):
        print_dbg(f"** Switch training to update_weight")                
        print_heading(f"{timestring()} - SWITCH TO WEIGHT TRAINING  urrent_iter: {current_iter}\n"
                      f"{' ':15s} current_iter_w: {current_iter_w}  batch_idx_w:{batch_idx_w}   weight_iter_alternate: {opt['train']['weight_iter_alternate']}\n"
                      f"{' ':15s} current_iter_a: {current_iter_a}  batch_idx_a:{batch_idx_a}   alpha_iter_alternate : {opt['train']['alpha_iter_alternate']} ",
                      verbose = True )       
        
        flag = 'update_w'
        environ.fix_alpha()
        environ.free_w(opt['fix_BN'])
        environ.decay_temperature()

        # print the distribution
        dists = environ.get_policy_prob()

        print(np.concatenate(dists, axis=-1))
        p_epoch += 1
        print(f"** p_epoch incremented: {p_epoch}")




In [ ]:
print(current_iter)

In [ ]:
# print(f" task1_logits: \n {environ.networks['mtl-net'].task1_logits.detach().cpu().numpy()} \n")
# print(f" task2_logits: \n {environ.networks['mtl-net'].task2_logits.detach().cpu().numpy()} \n")
# print(f" task3_logits: \n {environ.networks['mtl-net'].task3_logits.detach().cpu().numpy()} \n")
# print(f" task1 softmax: \n {softmax(environ.networks['mtl-net'].task1_logits.detach().cpu().numpy(), axis = -1)} \n")
# print(f" task2 softmax: \n {softmax(environ.networks['mtl-net'].task2_logits.detach().cpu().numpy(), axis = -1)} \n")
# print(f" task3 softmax: \n {softmax(environ.networks['mtl-net'].task3_logits.detach().cpu().numpy(), axis = -1)} \n")

In [ ]:
# for i in [1,2,3]:
#     task_pred = f"task{i}_pred"
#     task_logits = f"task{i}_logits"
#     policy_attr = f"policy{i}"
#     logits_attr = f"logit{i}"
#     print_heading(f"{task_pred}")
#     print(getattr(environ, task_pred))
#     print(policy_attr)
#     print(getattr(environ, policy_attr)) 
#     print(logits_attr)
#     print(getattr(environ, logits_attr)) 
#     print(task_logits)
#     print(getattr(environ.networks['mtl-net'], task_logits)) 

### Weight Training Dev

#### Weight training - prep

In [ ]:
# arch_parms = environ.networks['mtl-net'].named_parameters()
# print(arch_parms)
# for name, parm in arch_parms:
#     print(name, '    ',parm.requires_grad)
# print_underline('MTL3_Dev Policys', verbose = True)
# for i in   environ.networks['mtl-net'].policys:
#     print(i)

In [ ]:
print_heading(f"** {timestring()} - Training current iteration {current_iter}  flag: {flag} ", verbose = True)    

current_iter_w = 0 
current_iter_a = 0 
batch_idx_a = 0 
batch_idx_w = 0 

if flag_warmup:
    print_heading(f"** Set optimizer and scheduler to policy_learning = True", verbose = True)
    environ.define_optimizer(policy_learning=True)
    environ.define_scheduler(policy_learning=True)
    flag_warmup = False

if current_iter == opt['train']['warm_up_iters']:
    print_heading(f"** Switch from Warm Up training to Alternate training Weights & Policy \n"
                  f"   Take checkpoint and block gradient flow through Policy net", verbose=True)
    environ.save_checkpoint('warmup', current_iter)
    environ.fix_alpha()
    
# batch_enumerator1 = enumerate(train1_loader,1)  
# batch_enumerator2 = enumerate(train2_loader,1)  

train_total_epochs = 10

#### Weight training - main 

In [ ]:
print(f"opt['train']['print_freq']         {opt['train']['print_freq']}")
print(f"opt['train']['hard_sampling']      {opt['train']['hard_sampling']}")
print(f"opt['policy']                      {opt['policy']}")
print(f"opt['tasks']                       {opt['tasks']}")
print(f"weight_iter_alternate:             {opt['train']['weight_iter_alternate']}")
print(f"alpha_iter_alternate :             {opt['train']['alpha_iter_alternate']}")
print(f"current_iter                       {current_iter  }")
print(f"current_iter_w                     {current_iter_w}")
print(f"current_iter_a                     {current_iter_a}")
print(f"batch_idx_w                        {batch_idx_w}")
print(f"flag                               {flag          }")
print(f"train_total_epochs                 {train_total_epochs}") 

In [ ]:
##---------------------------------------------------------------     
## Weight / Policy Training
##--------------------------------------------------------------- 
# stop_iter = current_iter_w +  opt['train']['weight_iter_alternate']
# print(f" Current Weight iteration {current_iter_w} - Run  from {current_iter_w+1} to {stop_iter+1}")


In [ ]:
from tqdm.notebook import trange, tqdm

In [ ]:
# with tnrange(start_iter_t , stop_iter_t  , initial = start_iter_t , total = stop_iter_t, position=0, leave= True, desc="master") as t :
# with tqdm_notebook(total=train_total_epochs) as t:

In [ ]:
current_epoch = 0
main_iter_ctr = 0 
verbose = False
t = tqdm(total=train_total_epochs, desc=f" Alternate Weight/Policy training")

while current_epoch < train_total_epochs:
    current_epoch+=1
    t.update(1)

    #-----------------------------------------
    # Train & Update the network weights
    #-----------------------------------------
    if flag == 'update_w':
        current_iter_w  = 0 
        stop_iter_w =   opt['train']['weight_iter_alternate']

        with trange(+1, stop_iter_w+1 , initial = current_iter_w, total = stop_iter_w, 
                     position=0, leave= False, desc=f"Epoch {current_epoch} weight training") as t_weights :
            for current_iter_w in t_weights:    
                current_iter += 1

                start_time = time.time()
                environ.train()
                
#                 if batch_idx_w == len(train1_loader):
#                     print_dbg(f"  Reenumerate train1_loader -  index_w: {batch_idx_w}   len(train1_loader) = {len(train1_loader)} ", verbose)
#                     batch_enumerator1 = enumerate(train1_loader,1)    
                    
                batch = next(train1_loader)
                environ.set_inputs(batch, train1_loader.dataset.input_size)

                ##----------------------------------------------------------------------
                ## Set number of layers to train based on cirriculum_speed 
                ## and p_epoch (number of epochs of policy training)
                ## When curriculum_speed == 3, a num_train_layers is incremented 
                ## after completion of every 3 policy training epochs
                ##----------------------------------------------------------------------
                if opt['is_curriculum']:
                    num_train_layers = p_epoch // opt['curriculum_speed'] + 1
                else:
                    num_train_layers = None


#                 print_heading(f"{timestring()} CALL ENVIRON.OPTIMIZE()    current_iter: {current_iter}     flag: {flag}\n"
#                       f"{' ':10s} current_iter_w: {current_iter_w}  batch_idx_w:{batch_idx_w}   weight_iter_alternate: {opt['train']['weight_iter_alternate']}\n"
#                       f"{' ':10s} current_iter_a: {current_iter_a}  batch_idx_a:{batch_idx_a}   alpha_iter_alternate : {opt['train']['alpha_iter_alternate']} \n"                          
#                       f"{' ':10s} is_policy: {opt['policy']}     p_epoch: {p_epoch}       num_train_layers: {num_train_layers}", verbose = False) 

                environ.optimize(opt['lambdas'], 
                                 is_policy=opt['policy'], 
                                 flag=flag, 
                                 num_train_layers=num_train_layers,
                                 hard_sampling=opt['train']['hard_sampling'],
                                 verbose = False)

                t_weights.set_postfix({'iteration': current_iter, 'Loss': f"{environ.losses['total']['total'].item():.4f}" , 
                                       'row_ids':f"{batch['row_id'][0]}-{batch['row_id'][-1]}"})
                
                if should(current_iter, opt['train']['print_freq']):
                    environ.print_loss(current_iter, start_time, title = "Weight training iteration", verbose = True)
                    environ.resize_results()

#                 print_heading(f"{timestring()} - CONTINUE WEIGHT TRAINING   current_iter: {current_iter}\n"
#                   f" current_iter_w: {current_iter_w}  batch_idx_w:{batch_idx_w}   weight_iter_alternate: {opt['train']['weight_iter_alternate']}\n"
#                   f" current_iter_a: {current_iter_a}  batch_idx_a:{batch_idx_a}   alpha_iter_alternate : {opt['train']['alpha_iter_alternate']}",
#                   verbose = False)        

        #-------------------------------------------------------
        # validation process
        #------------------------------------------------------- 

#         if should(current_iter_w, opt['train']['weight_iter_alternate']): 

        if (current_iter_w >= stop_iter_w):
            environ.eval()
            print_dbg("++ Weight Training Validation  and then Switch to update_alpha", verbose = False)

            val_metrics = eval_dev(environ, 
                                  val_loader, 
                                  opt['tasks'], 
                                  policy=opt['policy'],
                                  num_train_layers=num_train_layers, 
                                  hard_sampling=opt['train']['hard_sampling'],
                                  eval_iter = -1)        

            if (verbose):
                for t_id, task in enumerate(environ.tasks):
                    task_key = f"task{t_id+1}"    
                    environ.print_metrics(current_iter, start_time, val_metrics[task_key]["classification_agg"], title='validation', verbose = verbose)        

            environ.save_checkpoint('latest', current_iter)

            #----------------------------------------------------------------------------------------------
            # if number of iterations completed after the warm up phase is greater than the number of 
            # (weight/policy alternations) x (cirriculum speed) x (number of layers to be policy trained)
            #
            # check metrics for improvement, and issue a checkpoint if necessary
            #----------------------------------------------------------------------------------------------



            if current_iter - opt['train']['warm_up_iters'] >= num_blocks * opt['curriculum_speed'] * \
                    (opt['train']['weight_iter_alternate'] + opt['train']['alpha_iter_alternate']):
                new_value = 0
#                 print_heading(f"  evaluate progress and make checkpoint if necessary." , verbose = True)
#                 print(f" current iter                                 : {current_iter} \n"
#                       f" opt['train']['warm_up_iters']                : {opt['train']['warm_up_iters']} \n"
#                       f" num_blocks                                   : {num_blocks} \n"
#                       f" opt['curriculum_speed']                      : {opt['curriculum_speed']}\n"
#                       f" opt['train']['weight_iter_alternate']        : {opt['train']['weight_iter_alternate']}\n"
#                       f" opt['train']['alpha_iter_alternate']         : {opt['train']['alpha_iter_alternate']}\n"
#                       f" alpha_iter_alternate + weight_iter_alternate : {opt['train']['weight_iter_alternate'] + opt['train']['alpha_iter_alternate']}\n"
#                       f" num_blks * curriculum_speed * (alpha_alternate + weight_alternate): "
#                       f" {num_blocks * opt['curriculum_speed'] * (opt['train']['weight_iter_alternate'] + opt['train']['alpha_iter_alternate'])} \n"

                print(f"  {current_iter - opt['train']['warm_up_iters']} IS GREATER THAN "
                       f" {num_blocks * opt['curriculum_speed'] * (opt['train']['weight_iter_alternate'] + opt['train']['alpha_iter_alternate'])} -- "
                       f"  evaluate progress and make checkpoint if necessary." )            
#               ## compare validation metrics against reference metrics.

#                 for k in refer_metrics.keys():
#                     if k in val_metrics.keys():
#                         for kk in val_metrics[k].keys():
#                             if not kk in refer_metrics[k].keys():
#                                 continue
#                             if (k == 'sn' and kk in ['Angle Mean', 'Angle Median']) or (
#                                     k == 'depth' and not kk.startswith('sigma')) or (kk == 'err'):
#                                 value = refer_metrics[k][kk] / val_metrics[k][kk]
#                             else:
#                                 value = val_metrics[k][kk] / refer_metrics[k][kk]
#                             value = value / len(list(set(val_metrics[k].keys()) & set(refer_metrics[k].keys())))
#                             new_value += value

#                 print('Best Value %.4f  New value: %.4f' % new_value)

                # if results have improved, save these results and issue a checkpoint

#                 if (new_value > best_value):
#                     print('Previous best iter: %d, best_value: %.4f' % (best_iter, best_value), best_metrics)
#                     best_value = new_value
#                     best_metrics = val_metrics
#                     best_iter = current_iter
#                     environ.save_checkpoint('best', current_iter)
#                     print('New      best iter: %d, best_value: %.4f' % (best_iter, best_value), best_metrics)                         
#                     print('Best Value %.4f  New value: %.4f' % new_value)

                # if results have improved, save these results and issue a checkpoint   

            environ.train()
            #-------------------------------------------------------
            # END validation process
            #-------------------------------------------------------       
            print_heading(f"{timestring()} - SWITCH TO ALPHA TRAINING    current_iter: {current_iter}\n"
              f" current_iter_w: {current_iter_w}  batch_idx_w:{batch_idx_w}   weight_iter_alternate: {opt['train']['weight_iter_alternate']}\n"
              f" current_iter_a: {current_iter_a}  batch_idx_a:{batch_idx_a}   alpha_iter_alternate : {opt['train']['alpha_iter_alternate']}",
              verbose = False)       
            flag = 'update_alpha'
            environ.fix_w()
            environ.free_alpha()
        #-------------------------------------------------------
        # end validation process
        #-------------------------------------------------------               


    #-----------------------------------------
    # Train & Update the  policy 
    #-----------------------------------------
    if flag == 'update_alpha':
        current_iter_a = 0
        stop_iter_a = opt['train']['alpha_iter_alternate']

        with trange( +1, stop_iter_a+1 , initial = 0, total = stop_iter_a, 
                     position=0, leave= False, desc=f"Epoch {current_epoch} policy training") as t_policy :
            for current_iter_a in t_policy:    
                current_iter += 1

#                 batch_idx_a, batch = next(batch_enumerator2)
                batch = next(train2_loader)
                environ.set_inputs(batch, train2_loader.dataset.input_size)

#                 if batch_idx_a == len(train2_loader):
#                     print_dbg(f" Re-enumerate train2_loader  batch_idx_a: {batch_idx_a}   len(train2_loader) = {len(train2_loader)}", verbose=False)                
#                     batch_enumerator2 = enumerate(train2_loader,1)        

#                 print_heading(f"{timestring()} - ENVIRON.OPTIMIZE()    flag: {flag}    current_iter: {current_iter}   \n"
#                               f" current_iter_w: {current_iter_w}  batch_idx_w:{batch_idx_w}   weight_iter_alternate: {opt['train']['weight_iter_alternate']}\n"
#                               f" current_iter_a: {current_iter_a}  batch_idx_a:{batch_idx_a}   alpha_iter_alternate : {opt['train']['alpha_iter_alternate']} \n"
#                               f" is_policy: {opt['policy']}   num_train_layers: {num_train_layers}  hard_sampling: {opt['train']['hard_sampling']}\n"
#                               f" is_curriculum: {opt['is_curriculum']}     curriculum_speed: {opt['curriculum_speed']}   p_epoch: {p_epoch}"
#                               , verbose = False) 

                if opt['is_curriculum']:
                    num_train_layers = (p_epoch // opt['curriculum_speed']) + 1
                else:
                    num_train_layers = None

                print_dbg(f" num_train_layers  : {num_train_layers}", verbose = False)


                environ.optimize(opt['lambdas'], 
                                 is_policy=opt['policy'], 
                                 flag=flag, 
                                 num_train_layers=num_train_layers,
                                 hard_sampling=opt['train']['hard_sampling'],
                                 verbose = False)
                
                t_policy.set_postfix({'iteration': current_iter, 'Loss': f"{environ.losses['total']['total'].item():.4f}" , 
                                      'row_ids':f"{batch['row_id'][0]}-{batch['row_id'][-1]}"})
                
                if should(current_iter, opt['train']['print_freq']):
                    environ.print_loss(current_iter, start_time, title = "Policy training iteration", verbose=True)
                    environ.resize_results()
                    # environ.visual_policy(current_iter)

#                 print_heading(f"{timestring()} - CONTINUE ALPHA TRAINING    current_iter: {current_iter}\n"
#                               f"{' ':15s} current_iter_w: {current_iter_w}  batch_idx_w:{batch_idx_w}   weight_iter_alternate: {opt['train']['weight_iter_alternate']}\n"
#                               f"{' ':15s} current_iter_a: {current_iter_a}  batch_idx_a:{batch_idx_a}   alpha_iter_alternate : {opt['train']['alpha_iter_alternate']} ", 
#                               verbose = False )      

        ## if (current_iter_a % alpha_iter_alternate) == 0 
#         if should(current_iter_a, opt['train']['alpha_iter_alternate']):
#         print(f" policy loop ended - current_iter_a: {current_iter_a}   stop_iter_a: {stop_iter_a}")
        if( current_iter_a >= stop_iter_a):            
#             print_heading(f"{timestring()} - SWITCH TO WEIGHT TRAINING  urrent_iter: {current_iter}\n"
#                           f"{' ':15s} current_iter_w: {current_iter_w}  batch_idx_w:{batch_idx_w}   weight_iter_alternate: {opt['train']['weight_iter_alternate']}\n"
#                           f"{' ':15s} current_iter_a: {current_iter_a}  batch_idx_a:{batch_idx_a}   alpha_iter_alternate : {opt['train']['alpha_iter_alternate']} ",
#                           verbose = False )       

            flag = 'update_w'
            environ.fix_alpha()
            environ.free_w(opt['fix_BN'])
            environ.decay_temperature()

            # print the distribution
            print_dbg(np.concatenate(environ.get_policy_prob(), axis=-1), verbose = False)
            
            p_epoch += 1
            print_dbg(f"** p_epoch incremented: {p_epoch}")

In [ ]:
print(f"{opt['train']['Lambda_sharing']:.5e}")

In [ ]:
print('Previous best iter: %d, best_value: %.4f' % (best_iter, best_value))
print(best_metrics)
best_value = new_value
best_metrics = val_metrics
best_iter = current_iter
environ.save_checkpoint('best', current_iter)
print('New best iter : %d, best_value: %.4f \n' % (best_iter, best_value))                         
print(best_metrics)

In [ ]:
# environ.losses['tasks'] = {'total' : torch.tensor(0.0, device  = environ.device, dtype=torch.float64)}
# environ.device

# print(val_metrics)
pp.pprint(environ.losses)
# environ.print_loss_2(current_iter, start_time, verbose=True)